In [ ]:
import pandas as pd                                                  # importing pandas library
import numpy as np                                                   # importing numpy library 
import missingno as msno                                             # importing missingno for finding the missing values
from sklearn.preprocessing import LabelEncoder                       # Importing labelEncoder for encoding
from sklearn.model_selection import train_test_split                 # Importing train_test_split for splitting dataset
from sklearn.preprocessing import MinMaxScaler                       # Importing for normalisation
from sklearn.neighbors import KNeighborsClassifier                   # Importing Modelling libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold          # Importing for k-fold cross validation
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('EEG_machinelearing_data.csv')     # Reading the Dataset and storing into a Dataframe named 'df'
df

### Data cleaning as a part of data preparation

In [ ]:
df.isnull()                                   # finding the values in the columns where there are missing values

In [ ]:
msno.matrix(df)             # Plotting the graph in order to visualise the missing values

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)                        # Finding the percentage of missing values so that if the percentage of missing value is greater than #10% so the column can be dropped.len() is used to find the length of the dataframe 
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})         

if percent_missing > 0.00:
    print(missing_value_df)

In [ ]:
missing_value_df                                       # printing the missing value percentage in the respective columns

In [ ]:
df.fillna(df.median().round(1), inplace=True)     # filling missing numerical values with median
print(df)

In [ ]:
df.isnull()                                 # checking if all numerical values have been replaced or not

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)                      
missing_value_df = pd.DataFrame({'column_name': df.columns,                 
                                 'percent_missing': percent_missing})   # 0 missing values percentage means that all missing numerical values have been filled 
                                                                         

In [ ]:
missing_value_df

In [ ]:
df.dropna(how = 'all', inplace = True)   # for dropping any columns having NaN but since there are no columns 

In [ ]:
column_with_nan = df.columns[df.isna().any()]   # Using this to check if there are any NaN values in the column

In [ ]:
# Using For loop to access each columns with NaN value
for column in column_with_nan:
    # Using .mean() in order to calculate the percentage of the NaN Values in the columns
    # It also drops the columns if the percentage of NaN values in the rows is more than 45% just to be safe in order to get rid of remaining NaN
    df = df.loc[:, df.isna().mean() <= 0.65]

In [ ]:
# Using this in order to check if there are any duplicated rows in the dataframe
df.duplicated()

In [ ]:
df  # printing the dataframe 'df' to see if there are any duplicated rows and as shown in the output,there are no duplicated rows 

In [ ]:
# dropping columns which are unnecessary
df1=df.drop(['specific.disorder'],axis=1)  # specific disorders column is dropped as classification will be done based on main disorders


In [ ]:
df2=df1.drop(['no.'], axis=1)   # Index number columns are dropped as they don't affect the classification of psychiatric disorders

In [ ]:
df3=df2.drop(['eeg.date'], axis=1)  # date on which eeg reading is taken also is dropped as it doesn't add any significance to the classification

In [ ]:
# Dropping other columns which are irrelevant as sex,IQ,age,education have no impact on the classification of psychiatric disorders based on EEG readings
df4=df3.drop(['sex'],axis=1)
df5=df4.drop(['IQ'],axis=1)
df6=df5.drop(['age'],axis=1)
df7=df6.drop(['education'],axis=1)


In [ ]:
df7  # Dsiplaying the new dataframe after dropping columns 

In [ ]:
df8 = df7.drop_duplicates()  # Earlier duplicated rows were seaarched to be dropped and now duplicate columns were searched to be dropped
df8
# Since columns which were not logically inconsistent were dropped and now it is just disorder and EEG readings so there is no need for further dropping columns which are logically inconsistent

In [ ]:
df8

In [ ]:
print(df8['disorder'].unique())    # using unique() to see the unique values in 'disorder' column


In [ ]:
# # Exploratory data analysis used to discover trends, patterns, or to check assumptions with the help of statistical summary and graphical representations

df8.describe()    # For descriptive statistics


In [ ]:
df8.median() # calculating the median of the dataframe

In [ ]:
df8['disorder'].mode()  # calculating the mode of the dataframe

In [ ]:
# Splits into Numeric Data
numeric_data = df8.select_dtypes(include=[np.number])
# Splits into Categorical Data
categorical_data = df8.select_dtypes(exclude=[np.number])


In [ ]:
# Storing the name of the numeric data column into a separate list
numeric_cols = numeric_data.columns.values

# Storing the name of the categorical data column into a separate list
categorical_cols = categorical_data.columns.values

In [ ]:
# Printing the column names of the numeric data
print(numeric_cols)

In [ ]:
# Printing the column names of the categorical data
print(categorical_cols)

In [ ]:
df8['disorder']

In [ ]:
numeric_cols


### visual representation of data (histograms,Q-Q plots and boxplots)

In [ ]:

for x in numeric_cols[:]:
    fig, ax = plt.subplots(1,1)
    ax.hist(numeric_data[x])                        # creating histograms in order to check the spread of the data
    ax.set_title("Histogram of "+x)
    ax.set_xlabel(x+" feature value")
    ax.set_ylabel("Frequency")
    plt.show()

In [ ]:
print(df8.columns.tolist())           # Converting columns into list for getting all variables at one place to be used in q-q plots plotting and further

In [ ]:
# plotting Q-Q plots for all the variables
import statsmodels.api as sm
arr3=['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.b.FP2', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.f.F4', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.o.Pz', 'AB.B.theta.p.P4', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.b.FP2', 'AB.C.alpha.c.F7', 'AB.C.alpha.d.F3', 'AB.C.alpha.e.Fz', 'AB.C.alpha.f.F4', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.b.FP2', 'AB.D.beta.c.F7', 'AB.D.beta.d.F3', 'AB.D.beta.e.Fz', 'AB.D.beta.f.F4', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.k.C4', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.p.P4', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.e.Fz', 'AB.E.highbeta.f.F4', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.p.P4', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.m.T5', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.d.F3.s.O2', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.m.T5', 'COH.B.theta.e.Fz.n.P3', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.p.P4', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.p.P4', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.q.T6', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.i.C3.s.O2', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.e.Fz', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.h.T3', 'COH.C.alpha.b.FP2.i.C3', 'COH.C.alpha.b.FP2.j.Cz', 'COH.C.alpha.b.FP2.k.C4', 'COH.C.alpha.b.FP2.l.T4', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.n.P3', 'COH.C.alpha.b.FP2.o.Pz', 'COH.C.alpha.b.FP2.p.P4', 'COH.C.alpha.b.FP2.q.T6', 'COH.C.alpha.b.FP2.r.O1', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.e.Fz', 'COH.C.alpha.c.F7.f.F4', 'COH.C.alpha.c.F7.g.F8', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.k.C4', 'COH.C.alpha.c.F7.l.T4', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.o.Pz', 'COH.C.alpha.c.F7.p.P4', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.c.F7.s.O2', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.g.F8', 'COH.C.alpha.d.F3.h.T3', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.d.F3.k.C4', 'COH.C.alpha.d.F3.l.T4', 'COH.C.alpha.d.F3.m.T5', 'COH.C.alpha.d.F3.n.P3', 'COH.C.alpha.d.F3.o.Pz', 'COH.C.alpha.d.F3.p.P4', 'COH.C.alpha.d.F3.q.T6', 'COH.C.alpha.d.F3.r.O1', 'COH.C.alpha.d.F3.s.O2', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.e.Fz.g.F8', 'COH.C.alpha.e.Fz.h.T3', 'COH.C.alpha.e.Fz.i.C3', 'COH.C.alpha.e.Fz.j.Cz', 'COH.C.alpha.e.Fz.k.C4', 'COH.C.alpha.e.Fz.l.T4', 'COH.C.alpha.e.Fz.m.T5', 'COH.C.alpha.e.Fz.n.P3', 'COH.C.alpha.e.Fz.o.Pz', 'COH.C.alpha.e.Fz.p.P4', 'COH.C.alpha.e.Fz.q.T6', 'COH.C.alpha.e.Fz.r.O1', 'COH.C.alpha.e.Fz.s.O2', 'COH.C.alpha.f.F4.g.F8', 'COH.C.alpha.f.F4.h.T3', 'COH.C.alpha.f.F4.i.C3', 'COH.C.alpha.f.F4.j.Cz', 'COH.C.alpha.f.F4.k.C4', 'COH.C.alpha.f.F4.l.T4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.n.P3', 'COH.C.alpha.f.F4.o.Pz', 'COH.C.alpha.f.F4.p.P4', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.h.T3', 'COH.C.alpha.g.F8.i.C3', 'COH.C.alpha.g.F8.j.Cz', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.o.Pz', 'COH.C.alpha.g.F8.p.P4', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.j.Cz', 'COH.C.alpha.h.T3.k.C4', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.l.T4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.p.P4', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.l.T4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.p.P4', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.p.P4', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.l.T4', 'COH.D.beta.d.F3.m.T5', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.d.F3.p.P4', 'COH.D.beta.d.F3.q.T6', 'COH.D.beta.d.F3.r.O1', 'COH.D.beta.d.F3.s.O2', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.e.Fz.l.T4', 'COH.D.beta.e.Fz.m.T5', 'COH.D.beta.e.Fz.n.P3', 'COH.D.beta.e.Fz.o.Pz', 'COH.D.beta.e.Fz.p.P4', 'COH.D.beta.e.Fz.q.T6', 'COH.D.beta.e.Fz.r.O1', 'COH.D.beta.e.Fz.s.O2', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.o.Pz', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.m.T5', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.g.F8.r.O1', 'COH.D.beta.g.F8.s.O2', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.q.T6', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.o.Pz', 'COH.E.highbeta.b.FP2.p.P4', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.b.FP2.s.O2', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.o.Pz', 'COH.E.highbeta.c.F7.p.P4', 'COH.E.highbeta.c.F7.q.T6', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.c.F7.s.O2', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.l.T4', 'COH.E.highbeta.d.F3.m.T5', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.d.F3.o.Pz', 'COH.E.highbeta.d.F3.p.P4', 'COH.E.highbeta.d.F3.q.T6', 'COH.E.highbeta.d.F3.r.O1', 'COH.E.highbeta.d.F3.s.O2', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.n.P3', 'COH.E.highbeta.e.Fz.o.Pz', 'COH.E.highbeta.e.Fz.p.P4', 'COH.E.highbeta.e.Fz.q.T6', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.e.Fz.s.O2', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.l.T4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.o.Pz', 'COH.E.highbeta.f.F4.p.P4', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.r.O1', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.p.P4', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.k.C4', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.q.T6', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.i.C3.s.O2', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.l.T4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.o.Pz', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.b.FP2', 'COH.F.gamma.a.FP1.c.F7', 'COH.F.gamma.a.FP1.d.F3', 'COH.F.gamma.a.FP1.e.Fz', 'COH.F.gamma.a.FP1.f.F4', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.j.Cz', 'COH.F.gamma.a.FP1.k.C4', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.o.Pz', 'COH.F.gamma.a.FP1.p.P4', 'COH.F.gamma.a.FP1.q.T6', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.d.F3', 'COH.F.gamma.b.FP2.e.Fz', 'COH.F.gamma.b.FP2.f.F4', 'COH.F.gamma.b.FP2.g.F8', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.j.Cz', 'COH.F.gamma.b.FP2.k.C4', 'COH.F.gamma.b.FP2.l.T4', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.b.FP2.n.P3', 'COH.F.gamma.b.FP2.o.Pz', 'COH.F.gamma.b.FP2.p.P4', 'COH.F.gamma.b.FP2.q.T6', 'COH.F.gamma.b.FP2.r.O1', 'COH.F.gamma.b.FP2.s.O2', 'COH.F.gamma.c.F7.d.F3', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.f.F4', 'COH.F.gamma.c.F7.g.F8', 'COH.F.gamma.c.F7.h.T3', 'COH.F.gamma.c.F7.i.C3', 'COH.F.gamma.c.F7.j.Cz', 'COH.F.gamma.c.F7.k.C4', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.c.F7.m.T5', 'COH.F.gamma.c.F7.n.P3', 'COH.F.gamma.c.F7.o.Pz', 'COH.F.gamma.c.F7.p.P4', 'COH.F.gamma.c.F7.q.T6', 'COH.F.gamma.c.F7.r.O1', 'COH.F.gamma.c.F7.s.O2', 'COH.F.gamma.d.F3.e.Fz', 'COH.F.gamma.d.F3.f.F4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.d.F3.h.T3', 'COH.F.gamma.d.F3.i.C3', 'COH.F.gamma.d.F3.j.Cz', 'COH.F.gamma.d.F3.k.C4', 'COH.F.gamma.d.F3.l.T4', 'COH.F.gamma.d.F3.m.T5', 'COH.F.gamma.d.F3.n.P3', 'COH.F.gamma.d.F3.o.Pz', 'COH.F.gamma.d.F3.p.P4', 'COH.F.gamma.d.F3.q.T6', 'COH.F.gamma.d.F3.r.O1', 'COH.F.gamma.d.F3.s.O2', 'COH.F.gamma.e.Fz.f.F4', 'COH.F.gamma.e.Fz.g.F8', 'COH.F.gamma.e.Fz.h.T3', 'COH.F.gamma.e.Fz.i.C3', 'COH.F.gamma.e.Fz.j.Cz', 'COH.F.gamma.e.Fz.k.C4', 'COH.F.gamma.e.Fz.l.T4', 'COH.F.gamma.e.Fz.m.T5', 'COH.F.gamma.e.Fz.n.P3', 'COH.F.gamma.e.Fz.o.Pz', 'COH.F.gamma.e.Fz.p.P4', 'COH.F.gamma.e.Fz.q.T6', 'COH.F.gamma.e.Fz.r.O1', 'COH.F.gamma.e.Fz.s.O2', 'COH.F.gamma.f.F4.g.F8', 'COH.F.gamma.f.F4.h.T3', 'COH.F.gamma.f.F4.i.C3', 'COH.F.gamma.f.F4.j.Cz', 'COH.F.gamma.f.F4.k.C4', 'COH.F.gamma.f.F4.l.T4', 'COH.F.gamma.f.F4.m.T5', 'COH.F.gamma.f.F4.n.P3', 'COH.F.gamma.f.F4.o.Pz', 'COH.F.gamma.f.F4.p.P4', 'COH.F.gamma.f.F4.q.T6', 'COH.F.gamma.f.F4.r.O1', 'COH.F.gamma.f.F4.s.O2', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.i.C3', 'COH.F.gamma.g.F8.j.Cz', 'COH.F.gamma.g.F8.k.C4', 'COH.F.gamma.g.F8.l.T4', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.o.Pz', 'COH.F.gamma.g.F8.p.P4', 'COH.F.gamma.g.F8.q.T6', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.i.C3', 'COH.F.gamma.h.T3.j.Cz', 'COH.F.gamma.h.T3.k.C4', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.m.T5', 'COH.F.gamma.h.T3.n.P3', 'COH.F.gamma.h.T3.o.Pz', 'COH.F.gamma.h.T3.p.P4', 'COH.F.gamma.h.T3.q.T6', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.j.Cz', 'COH.F.gamma.i.C3.k.C4', 'COH.F.gamma.i.C3.l.T4', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.n.P3', 'COH.F.gamma.i.C3.o.Pz', 'COH.F.gamma.i.C3.p.P4', 'COH.F.gamma.i.C3.q.T6', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.i.C3.s.O2', 'COH.F.gamma.j.Cz.k.C4', 'COH.F.gamma.j.Cz.l.T4', 'COH.F.gamma.j.Cz.m.T5', 'COH.F.gamma.j.Cz.n.P3', 'COH.F.gamma.j.Cz.o.Pz', 'COH.F.gamma.j.Cz.p.P4', 'COH.F.gamma.j.Cz.q.T6', 'COH.F.gamma.j.Cz.r.O1', 'COH.F.gamma.j.Cz.s.O2', 'COH.F.gamma.k.C4.l.T4', 'COH.F.gamma.k.C4.m.T5', 'COH.F.gamma.k.C4.n.P3', 'COH.F.gamma.k.C4.o.Pz', 'COH.F.gamma.k.C4.p.P4', 'COH.F.gamma.k.C4.q.T6', 'COH.F.gamma.k.C4.r.O1', 'COH.F.gamma.k.C4.s.O2', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.n.P3', 'COH.F.gamma.l.T4.o.Pz', 'COH.F.gamma.l.T4.p.P4', 'COH.F.gamma.l.T4.q.T6', 'COH.F.gamma.l.T4.r.O1', 'COH.F.gamma.l.T4.s.O2', 'COH.F.gamma.m.T5.n.P3', 'COH.F.gamma.m.T5.o.Pz', 'COH.F.gamma.m.T5.p.P4', 'COH.F.gamma.m.T5.q.T6', 'COH.F.gamma.m.T5.r.O1', 'COH.F.gamma.m.T5.s.O2', 'COH.F.gamma.n.P3.o.Pz', 'COH.F.gamma.n.P3.p.P4', 'COH.F.gamma.n.P3.q.T6', 'COH.F.gamma.n.P3.r.O1', 'COH.F.gamma.n.P3.s.O2', 'COH.F.gamma.o.Pz.p.P4', 'COH.F.gamma.o.Pz.q.T6', 'COH.F.gamma.o.Pz.r.O1', 'COH.F.gamma.o.Pz.s.O2', 'COH.F.gamma.p.P4.q.T6', 'COH.F.gamma.p.P4.r.O1', 'COH.F.gamma.p.P4.s.O2', 'COH.F.gamma.q.T6.r.O1', 'COH.F.gamma.q.T6.s.O2', 'COH.F.gamma.r.O1.s.O2']
for i in arr3:
     fig=sm.qqplot(numeric_data[i],line='45',xlabel="Theoretical "+i,ylabel=i+" feature value in the dataset") 
     plt.show()

In [ ]:
# Plotting boxplots as a part of graphical reprsentation of data

arr2=['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.b.FP2', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.f.F4', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.o.Pz', 'AB.B.theta.p.P4', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.b.FP2', 'AB.C.alpha.c.F7', 'AB.C.alpha.d.F3', 'AB.C.alpha.e.Fz', 'AB.C.alpha.f.F4', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.b.FP2', 'AB.D.beta.c.F7', 'AB.D.beta.d.F3', 'AB.D.beta.e.Fz', 'AB.D.beta.f.F4', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.k.C4', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.p.P4', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.e.Fz', 'AB.E.highbeta.f.F4', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.p.P4', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.m.T5', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.d.F3.s.O2', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.m.T5', 'COH.B.theta.e.Fz.n.P3', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.p.P4', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.p.P4', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.q.T6', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.i.C3.s.O2', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.e.Fz', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.h.T3', 'COH.C.alpha.b.FP2.i.C3', 'COH.C.alpha.b.FP2.j.Cz', 'COH.C.alpha.b.FP2.k.C4', 'COH.C.alpha.b.FP2.l.T4', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.n.P3', 'COH.C.alpha.b.FP2.o.Pz', 'COH.C.alpha.b.FP2.p.P4', 'COH.C.alpha.b.FP2.q.T6', 'COH.C.alpha.b.FP2.r.O1', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.e.Fz', 'COH.C.alpha.c.F7.f.F4', 'COH.C.alpha.c.F7.g.F8', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.k.C4', 'COH.C.alpha.c.F7.l.T4', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.o.Pz', 'COH.C.alpha.c.F7.p.P4', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.c.F7.s.O2', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.g.F8', 'COH.C.alpha.d.F3.h.T3', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.d.F3.k.C4', 'COH.C.alpha.d.F3.l.T4', 'COH.C.alpha.d.F3.m.T5', 'COH.C.alpha.d.F3.n.P3', 'COH.C.alpha.d.F3.o.Pz', 'COH.C.alpha.d.F3.p.P4', 'COH.C.alpha.d.F3.q.T6', 'COH.C.alpha.d.F3.r.O1', 'COH.C.alpha.d.F3.s.O2', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.e.Fz.g.F8', 'COH.C.alpha.e.Fz.h.T3', 'COH.C.alpha.e.Fz.i.C3', 'COH.C.alpha.e.Fz.j.Cz', 'COH.C.alpha.e.Fz.k.C4', 'COH.C.alpha.e.Fz.l.T4', 'COH.C.alpha.e.Fz.m.T5', 'COH.C.alpha.e.Fz.n.P3', 'COH.C.alpha.e.Fz.o.Pz', 'COH.C.alpha.e.Fz.p.P4', 'COH.C.alpha.e.Fz.q.T6', 'COH.C.alpha.e.Fz.r.O1', 'COH.C.alpha.e.Fz.s.O2', 'COH.C.alpha.f.F4.g.F8', 'COH.C.alpha.f.F4.h.T3', 'COH.C.alpha.f.F4.i.C3', 'COH.C.alpha.f.F4.j.Cz', 'COH.C.alpha.f.F4.k.C4', 'COH.C.alpha.f.F4.l.T4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.n.P3', 'COH.C.alpha.f.F4.o.Pz', 'COH.C.alpha.f.F4.p.P4', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.h.T3', 'COH.C.alpha.g.F8.i.C3', 'COH.C.alpha.g.F8.j.Cz', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.o.Pz', 'COH.C.alpha.g.F8.p.P4', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.j.Cz', 'COH.C.alpha.h.T3.k.C4', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.l.T4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.p.P4', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.l.T4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.p.P4', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.p.P4', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.l.T4', 'COH.D.beta.d.F3.m.T5', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.d.F3.p.P4', 'COH.D.beta.d.F3.q.T6', 'COH.D.beta.d.F3.r.O1', 'COH.D.beta.d.F3.s.O2', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.e.Fz.l.T4', 'COH.D.beta.e.Fz.m.T5', 'COH.D.beta.e.Fz.n.P3', 'COH.D.beta.e.Fz.o.Pz', 'COH.D.beta.e.Fz.p.P4', 'COH.D.beta.e.Fz.q.T6', 'COH.D.beta.e.Fz.r.O1', 'COH.D.beta.e.Fz.s.O2', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.o.Pz', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.m.T5', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.g.F8.r.O1', 'COH.D.beta.g.F8.s.O2', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.q.T6', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.o.Pz', 'COH.E.highbeta.b.FP2.p.P4', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.b.FP2.s.O2', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.o.Pz', 'COH.E.highbeta.c.F7.p.P4', 'COH.E.highbeta.c.F7.q.T6', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.c.F7.s.O2', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.l.T4', 'COH.E.highbeta.d.F3.m.T5', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.d.F3.o.Pz', 'COH.E.highbeta.d.F3.p.P4', 'COH.E.highbeta.d.F3.q.T6', 'COH.E.highbeta.d.F3.r.O1', 'COH.E.highbeta.d.F3.s.O2', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.n.P3', 'COH.E.highbeta.e.Fz.o.Pz', 'COH.E.highbeta.e.Fz.p.P4', 'COH.E.highbeta.e.Fz.q.T6', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.e.Fz.s.O2', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.l.T4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.o.Pz', 'COH.E.highbeta.f.F4.p.P4', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.r.O1', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.p.P4', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.k.C4', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.q.T6', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.i.C3.s.O2', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.l.T4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.o.Pz', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.b.FP2', 'COH.F.gamma.a.FP1.c.F7', 'COH.F.gamma.a.FP1.d.F3', 'COH.F.gamma.a.FP1.e.Fz', 'COH.F.gamma.a.FP1.f.F4', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.j.Cz', 'COH.F.gamma.a.FP1.k.C4', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.o.Pz', 'COH.F.gamma.a.FP1.p.P4', 'COH.F.gamma.a.FP1.q.T6', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.d.F3', 'COH.F.gamma.b.FP2.e.Fz', 'COH.F.gamma.b.FP2.f.F4', 'COH.F.gamma.b.FP2.g.F8', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.j.Cz', 'COH.F.gamma.b.FP2.k.C4', 'COH.F.gamma.b.FP2.l.T4', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.b.FP2.n.P3', 'COH.F.gamma.b.FP2.o.Pz', 'COH.F.gamma.b.FP2.p.P4', 'COH.F.gamma.b.FP2.q.T6', 'COH.F.gamma.b.FP2.r.O1', 'COH.F.gamma.b.FP2.s.O2', 'COH.F.gamma.c.F7.d.F3', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.f.F4', 'COH.F.gamma.c.F7.g.F8', 'COH.F.gamma.c.F7.h.T3', 'COH.F.gamma.c.F7.i.C3', 'COH.F.gamma.c.F7.j.Cz', 'COH.F.gamma.c.F7.k.C4', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.c.F7.m.T5', 'COH.F.gamma.c.F7.n.P3', 'COH.F.gamma.c.F7.o.Pz', 'COH.F.gamma.c.F7.p.P4', 'COH.F.gamma.c.F7.q.T6', 'COH.F.gamma.c.F7.r.O1', 'COH.F.gamma.c.F7.s.O2', 'COH.F.gamma.d.F3.e.Fz', 'COH.F.gamma.d.F3.f.F4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.d.F3.h.T3', 'COH.F.gamma.d.F3.i.C3', 'COH.F.gamma.d.F3.j.Cz', 'COH.F.gamma.d.F3.k.C4', 'COH.F.gamma.d.F3.l.T4', 'COH.F.gamma.d.F3.m.T5', 'COH.F.gamma.d.F3.n.P3', 'COH.F.gamma.d.F3.o.Pz', 'COH.F.gamma.d.F3.p.P4', 'COH.F.gamma.d.F3.q.T6', 'COH.F.gamma.d.F3.r.O1', 'COH.F.gamma.d.F3.s.O2', 'COH.F.gamma.e.Fz.f.F4', 'COH.F.gamma.e.Fz.g.F8', 'COH.F.gamma.e.Fz.h.T3', 'COH.F.gamma.e.Fz.i.C3', 'COH.F.gamma.e.Fz.j.Cz', 'COH.F.gamma.e.Fz.k.C4', 'COH.F.gamma.e.Fz.l.T4', 'COH.F.gamma.e.Fz.m.T5', 'COH.F.gamma.e.Fz.n.P3', 'COH.F.gamma.e.Fz.o.Pz', 'COH.F.gamma.e.Fz.p.P4', 'COH.F.gamma.e.Fz.q.T6', 'COH.F.gamma.e.Fz.r.O1', 'COH.F.gamma.e.Fz.s.O2', 'COH.F.gamma.f.F4.g.F8', 'COH.F.gamma.f.F4.h.T3', 'COH.F.gamma.f.F4.i.C3', 'COH.F.gamma.f.F4.j.Cz', 'COH.F.gamma.f.F4.k.C4', 'COH.F.gamma.f.F4.l.T4', 'COH.F.gamma.f.F4.m.T5', 'COH.F.gamma.f.F4.n.P3', 'COH.F.gamma.f.F4.o.Pz', 'COH.F.gamma.f.F4.p.P4', 'COH.F.gamma.f.F4.q.T6', 'COH.F.gamma.f.F4.r.O1', 'COH.F.gamma.f.F4.s.O2', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.i.C3', 'COH.F.gamma.g.F8.j.Cz', 'COH.F.gamma.g.F8.k.C4', 'COH.F.gamma.g.F8.l.T4', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.o.Pz', 'COH.F.gamma.g.F8.p.P4', 'COH.F.gamma.g.F8.q.T6', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.i.C3', 'COH.F.gamma.h.T3.j.Cz', 'COH.F.gamma.h.T3.k.C4', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.m.T5', 'COH.F.gamma.h.T3.n.P3', 'COH.F.gamma.h.T3.o.Pz', 'COH.F.gamma.h.T3.p.P4', 'COH.F.gamma.h.T3.q.T6', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.j.Cz', 'COH.F.gamma.i.C3.k.C4', 'COH.F.gamma.i.C3.l.T4', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.n.P3', 'COH.F.gamma.i.C3.o.Pz', 'COH.F.gamma.i.C3.p.P4', 'COH.F.gamma.i.C3.q.T6', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.i.C3.s.O2', 'COH.F.gamma.j.Cz.k.C4', 'COH.F.gamma.j.Cz.l.T4', 'COH.F.gamma.j.Cz.m.T5', 'COH.F.gamma.j.Cz.n.P3', 'COH.F.gamma.j.Cz.o.Pz', 'COH.F.gamma.j.Cz.p.P4', 'COH.F.gamma.j.Cz.q.T6', 'COH.F.gamma.j.Cz.r.O1', 'COH.F.gamma.j.Cz.s.O2', 'COH.F.gamma.k.C4.l.T4', 'COH.F.gamma.k.C4.m.T5', 'COH.F.gamma.k.C4.n.P3', 'COH.F.gamma.k.C4.o.Pz', 'COH.F.gamma.k.C4.p.P4', 'COH.F.gamma.k.C4.q.T6', 'COH.F.gamma.k.C4.r.O1', 'COH.F.gamma.k.C4.s.O2', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.n.P3', 'COH.F.gamma.l.T4.o.Pz', 'COH.F.gamma.l.T4.p.P4', 'COH.F.gamma.l.T4.q.T6', 'COH.F.gamma.l.T4.r.O1', 'COH.F.gamma.l.T4.s.O2', 'COH.F.gamma.m.T5.n.P3', 'COH.F.gamma.m.T5.o.Pz', 'COH.F.gamma.m.T5.p.P4', 'COH.F.gamma.m.T5.q.T6', 'COH.F.gamma.m.T5.r.O1', 'COH.F.gamma.m.T5.s.O2', 'COH.F.gamma.n.P3.o.Pz', 'COH.F.gamma.n.P3.p.P4', 'COH.F.gamma.n.P3.q.T6', 'COH.F.gamma.n.P3.r.O1', 'COH.F.gamma.n.P3.s.O2', 'COH.F.gamma.o.Pz.p.P4', 'COH.F.gamma.o.Pz.q.T6', 'COH.F.gamma.o.Pz.r.O1', 'COH.F.gamma.o.Pz.s.O2', 'COH.F.gamma.p.P4.q.T6', 'COH.F.gamma.p.P4.r.O1', 'COH.F.gamma.p.P4.s.O2', 'COH.F.gamma.q.T6.r.O1', 'COH.F.gamma.q.T6.s.O2', 'COH.F.gamma.r.O1.s.O2']
for i in arr2:
    cat_1 = df8[df8['disorder']=='Addictive disorder'][i]
    cat_2 = df8[df8['disorder']=='Trauma and stress related disorder'][i]
    cat_3 = df8[df8['disorder']=='Mood disorder'][i]
    cat_4 = df8[df8['disorder']=='Healthy control'][i]
    cat_5 = df8[df8['disorder']=='Obsessive compulsive disorder'][i]
    cat_6 = df8[df8['disorder']=='Schizophrenia'][i]
    cat_7 = df8[df8['disorder']=='Anxiety disorder'][i]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Boxplot of "+i)
    ax.boxplot([cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7], labels=['Addictive', 'Trauma', 'Mood', 'Healthy', 'OCD', 'Schizo', 'Anxiety'])
    ax.set_ylabel(i+" variable values")
    

### Detection and treatment of outliers

In [ ]:
#Detection and treatment of outliers using the percentile function as it is evident in the boxplot that there are outliers present in the dataset

for x in numeric_cols:
    q75,q25=np.percentile(df8.loc[:,x],[75,25])  # calculating the value of Q1 and Q3
    intr_qr=q75-q25
    max=q75+(1.5*intr_qr)
    min=q25-(1.5*intr_qr)
    median = df8.loc[df[x]<min, x].median()        
    df8.loc[df8[x]> max, x] = np.nan
    df8.fillna(median,inplace=True)          # Replacing outliers with median
   

In [ ]:
df8.isnull().sum()   # Checking the sum of null values or missing values

In [ ]:
# dropping the null values using pandas.dataframe.dropna() function if any

df8=df8.dropna(axis=0)

In [ ]:
df8.isnull().sum() # Now again checking the sum of null values to validate that all the outliers have been imputed and there is no null values

In [ ]:
df8      # Displaying the dataframe after imputing outliers with median

In [ ]:
# Again plotting the boxplot to see if the situation of outliers is better or not
arr2=['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.b.FP2', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.f.F4', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.o.Pz', 'AB.B.theta.p.P4', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.b.FP2', 'AB.C.alpha.c.F7', 'AB.C.alpha.d.F3', 'AB.C.alpha.e.Fz', 'AB.C.alpha.f.F4', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.b.FP2', 'AB.D.beta.c.F7', 'AB.D.beta.d.F3', 'AB.D.beta.e.Fz', 'AB.D.beta.f.F4', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.k.C4', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.p.P4', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.e.Fz', 'AB.E.highbeta.f.F4', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.p.P4', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.m.T5', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.d.F3.s.O2', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.m.T5', 'COH.B.theta.e.Fz.n.P3', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.p.P4', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.p.P4', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.q.T6', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.i.C3.s.O2', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.e.Fz', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.h.T3', 'COH.C.alpha.b.FP2.i.C3', 'COH.C.alpha.b.FP2.j.Cz', 'COH.C.alpha.b.FP2.k.C4', 'COH.C.alpha.b.FP2.l.T4', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.n.P3', 'COH.C.alpha.b.FP2.o.Pz', 'COH.C.alpha.b.FP2.p.P4', 'COH.C.alpha.b.FP2.q.T6', 'COH.C.alpha.b.FP2.r.O1', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.e.Fz', 'COH.C.alpha.c.F7.f.F4', 'COH.C.alpha.c.F7.g.F8', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.k.C4', 'COH.C.alpha.c.F7.l.T4', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.o.Pz', 'COH.C.alpha.c.F7.p.P4', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.c.F7.s.O2', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.g.F8', 'COH.C.alpha.d.F3.h.T3', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.d.F3.k.C4', 'COH.C.alpha.d.F3.l.T4', 'COH.C.alpha.d.F3.m.T5', 'COH.C.alpha.d.F3.n.P3', 'COH.C.alpha.d.F3.o.Pz', 'COH.C.alpha.d.F3.p.P4', 'COH.C.alpha.d.F3.q.T6', 'COH.C.alpha.d.F3.r.O1', 'COH.C.alpha.d.F3.s.O2', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.e.Fz.g.F8', 'COH.C.alpha.e.Fz.h.T3', 'COH.C.alpha.e.Fz.i.C3', 'COH.C.alpha.e.Fz.j.Cz', 'COH.C.alpha.e.Fz.k.C4', 'COH.C.alpha.e.Fz.l.T4', 'COH.C.alpha.e.Fz.m.T5', 'COH.C.alpha.e.Fz.n.P3', 'COH.C.alpha.e.Fz.o.Pz', 'COH.C.alpha.e.Fz.p.P4', 'COH.C.alpha.e.Fz.q.T6', 'COH.C.alpha.e.Fz.r.O1', 'COH.C.alpha.e.Fz.s.O2', 'COH.C.alpha.f.F4.g.F8', 'COH.C.alpha.f.F4.h.T3', 'COH.C.alpha.f.F4.i.C3', 'COH.C.alpha.f.F4.j.Cz', 'COH.C.alpha.f.F4.k.C4', 'COH.C.alpha.f.F4.l.T4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.n.P3', 'COH.C.alpha.f.F4.o.Pz', 'COH.C.alpha.f.F4.p.P4', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.h.T3', 'COH.C.alpha.g.F8.i.C3', 'COH.C.alpha.g.F8.j.Cz', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.o.Pz', 'COH.C.alpha.g.F8.p.P4', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.j.Cz', 'COH.C.alpha.h.T3.k.C4', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.l.T4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.p.P4', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.l.T4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.p.P4', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.p.P4', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.l.T4', 'COH.D.beta.d.F3.m.T5', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.d.F3.p.P4', 'COH.D.beta.d.F3.q.T6', 'COH.D.beta.d.F3.r.O1', 'COH.D.beta.d.F3.s.O2', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.e.Fz.l.T4', 'COH.D.beta.e.Fz.m.T5', 'COH.D.beta.e.Fz.n.P3', 'COH.D.beta.e.Fz.o.Pz', 'COH.D.beta.e.Fz.p.P4', 'COH.D.beta.e.Fz.q.T6', 'COH.D.beta.e.Fz.r.O1', 'COH.D.beta.e.Fz.s.O2', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.o.Pz', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.m.T5', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.g.F8.r.O1', 'COH.D.beta.g.F8.s.O2', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.q.T6', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.o.Pz', 'COH.E.highbeta.b.FP2.p.P4', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.b.FP2.s.O2', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.o.Pz', 'COH.E.highbeta.c.F7.p.P4', 'COH.E.highbeta.c.F7.q.T6', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.c.F7.s.O2', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.l.T4', 'COH.E.highbeta.d.F3.m.T5', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.d.F3.o.Pz', 'COH.E.highbeta.d.F3.p.P4', 'COH.E.highbeta.d.F3.q.T6', 'COH.E.highbeta.d.F3.r.O1', 'COH.E.highbeta.d.F3.s.O2', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.n.P3', 'COH.E.highbeta.e.Fz.o.Pz', 'COH.E.highbeta.e.Fz.p.P4', 'COH.E.highbeta.e.Fz.q.T6', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.e.Fz.s.O2', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.l.T4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.o.Pz', 'COH.E.highbeta.f.F4.p.P4', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.r.O1', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.p.P4', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.k.C4', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.q.T6', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.i.C3.s.O2', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.l.T4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.o.Pz', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.b.FP2', 'COH.F.gamma.a.FP1.c.F7', 'COH.F.gamma.a.FP1.d.F3', 'COH.F.gamma.a.FP1.e.Fz', 'COH.F.gamma.a.FP1.f.F4', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.j.Cz', 'COH.F.gamma.a.FP1.k.C4', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.o.Pz', 'COH.F.gamma.a.FP1.p.P4', 'COH.F.gamma.a.FP1.q.T6', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.d.F3', 'COH.F.gamma.b.FP2.e.Fz', 'COH.F.gamma.b.FP2.f.F4', 'COH.F.gamma.b.FP2.g.F8', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.j.Cz', 'COH.F.gamma.b.FP2.k.C4', 'COH.F.gamma.b.FP2.l.T4', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.b.FP2.n.P3', 'COH.F.gamma.b.FP2.o.Pz', 'COH.F.gamma.b.FP2.p.P4', 'COH.F.gamma.b.FP2.q.T6', 'COH.F.gamma.b.FP2.r.O1', 'COH.F.gamma.b.FP2.s.O2', 'COH.F.gamma.c.F7.d.F3', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.f.F4', 'COH.F.gamma.c.F7.g.F8', 'COH.F.gamma.c.F7.h.T3', 'COH.F.gamma.c.F7.i.C3', 'COH.F.gamma.c.F7.j.Cz', 'COH.F.gamma.c.F7.k.C4', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.c.F7.m.T5', 'COH.F.gamma.c.F7.n.P3', 'COH.F.gamma.c.F7.o.Pz', 'COH.F.gamma.c.F7.p.P4', 'COH.F.gamma.c.F7.q.T6', 'COH.F.gamma.c.F7.r.O1', 'COH.F.gamma.c.F7.s.O2', 'COH.F.gamma.d.F3.e.Fz', 'COH.F.gamma.d.F3.f.F4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.d.F3.h.T3', 'COH.F.gamma.d.F3.i.C3', 'COH.F.gamma.d.F3.j.Cz', 'COH.F.gamma.d.F3.k.C4', 'COH.F.gamma.d.F3.l.T4', 'COH.F.gamma.d.F3.m.T5', 'COH.F.gamma.d.F3.n.P3', 'COH.F.gamma.d.F3.o.Pz', 'COH.F.gamma.d.F3.p.P4', 'COH.F.gamma.d.F3.q.T6', 'COH.F.gamma.d.F3.r.O1', 'COH.F.gamma.d.F3.s.O2', 'COH.F.gamma.e.Fz.f.F4', 'COH.F.gamma.e.Fz.g.F8', 'COH.F.gamma.e.Fz.h.T3', 'COH.F.gamma.e.Fz.i.C3', 'COH.F.gamma.e.Fz.j.Cz', 'COH.F.gamma.e.Fz.k.C4', 'COH.F.gamma.e.Fz.l.T4', 'COH.F.gamma.e.Fz.m.T5', 'COH.F.gamma.e.Fz.n.P3', 'COH.F.gamma.e.Fz.o.Pz', 'COH.F.gamma.e.Fz.p.P4', 'COH.F.gamma.e.Fz.q.T6', 'COH.F.gamma.e.Fz.r.O1', 'COH.F.gamma.e.Fz.s.O2', 'COH.F.gamma.f.F4.g.F8', 'COH.F.gamma.f.F4.h.T3', 'COH.F.gamma.f.F4.i.C3', 'COH.F.gamma.f.F4.j.Cz', 'COH.F.gamma.f.F4.k.C4', 'COH.F.gamma.f.F4.l.T4', 'COH.F.gamma.f.F4.m.T5', 'COH.F.gamma.f.F4.n.P3', 'COH.F.gamma.f.F4.o.Pz', 'COH.F.gamma.f.F4.p.P4', 'COH.F.gamma.f.F4.q.T6', 'COH.F.gamma.f.F4.r.O1', 'COH.F.gamma.f.F4.s.O2', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.i.C3', 'COH.F.gamma.g.F8.j.Cz', 'COH.F.gamma.g.F8.k.C4', 'COH.F.gamma.g.F8.l.T4', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.o.Pz', 'COH.F.gamma.g.F8.p.P4', 'COH.F.gamma.g.F8.q.T6', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.i.C3', 'COH.F.gamma.h.T3.j.Cz', 'COH.F.gamma.h.T3.k.C4', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.m.T5', 'COH.F.gamma.h.T3.n.P3', 'COH.F.gamma.h.T3.o.Pz', 'COH.F.gamma.h.T3.p.P4', 'COH.F.gamma.h.T3.q.T6', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.j.Cz', 'COH.F.gamma.i.C3.k.C4', 'COH.F.gamma.i.C3.l.T4', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.n.P3', 'COH.F.gamma.i.C3.o.Pz', 'COH.F.gamma.i.C3.p.P4', 'COH.F.gamma.i.C3.q.T6', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.i.C3.s.O2', 'COH.F.gamma.j.Cz.k.C4', 'COH.F.gamma.j.Cz.l.T4', 'COH.F.gamma.j.Cz.m.T5', 'COH.F.gamma.j.Cz.n.P3', 'COH.F.gamma.j.Cz.o.Pz', 'COH.F.gamma.j.Cz.p.P4', 'COH.F.gamma.j.Cz.q.T6', 'COH.F.gamma.j.Cz.r.O1', 'COH.F.gamma.j.Cz.s.O2', 'COH.F.gamma.k.C4.l.T4', 'COH.F.gamma.k.C4.m.T5', 'COH.F.gamma.k.C4.n.P3', 'COH.F.gamma.k.C4.o.Pz', 'COH.F.gamma.k.C4.p.P4', 'COH.F.gamma.k.C4.q.T6', 'COH.F.gamma.k.C4.r.O1', 'COH.F.gamma.k.C4.s.O2', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.n.P3', 'COH.F.gamma.l.T4.o.Pz', 'COH.F.gamma.l.T4.p.P4', 'COH.F.gamma.l.T4.q.T6', 'COH.F.gamma.l.T4.r.O1', 'COH.F.gamma.l.T4.s.O2', 'COH.F.gamma.m.T5.n.P3', 'COH.F.gamma.m.T5.o.Pz', 'COH.F.gamma.m.T5.p.P4', 'COH.F.gamma.m.T5.q.T6', 'COH.F.gamma.m.T5.r.O1', 'COH.F.gamma.m.T5.s.O2', 'COH.F.gamma.n.P3.o.Pz', 'COH.F.gamma.n.P3.p.P4', 'COH.F.gamma.n.P3.q.T6', 'COH.F.gamma.n.P3.r.O1', 'COH.F.gamma.n.P3.s.O2', 'COH.F.gamma.o.Pz.p.P4', 'COH.F.gamma.o.Pz.q.T6', 'COH.F.gamma.o.Pz.r.O1', 'COH.F.gamma.o.Pz.s.O2', 'COH.F.gamma.p.P4.q.T6', 'COH.F.gamma.p.P4.r.O1', 'COH.F.gamma.p.P4.s.O2', 'COH.F.gamma.q.T6.r.O1', 'COH.F.gamma.q.T6.s.O2', 'COH.F.gamma.r.O1.s.O2']
for i in arr2:
    cat_1 = df8[df8['disorder']=='Addictive disorder'][i]
    cat_2 = df8[df8['disorder']=='Trauma and stress related disorder'][i]
    cat_3 = df8[df8['disorder']=='Mood disorder'][i]
    cat_4 = df8[df8['disorder']=='Healthy control'][i]
    cat_5 = df8[df8['disorder']=='Obsessive compulsive disorder'][i]
    cat_6 = df8[df8['disorder']=='Schizophrenia'][i]
    cat_7 = df8[df8['disorder']=='Anxiety disorder'][i]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Boxplot of "+i)
    ax.boxplot([cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7], labels=['Addictive', 'Trauma', 'Mood', 'Healthy', 'OCD', 'Schizo', 'Anxiety'])
    ax.set_ylabel(i+" variable values")

In [ ]:
# Splits into Numeric Data
numeric_data1 = df8.select_dtypes(include=[np.number])

In [ ]:
# again plotting Q-Q plots for all the variables
import statsmodels.api as sm
arr3=['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.b.FP2', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.f.F4', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.o.Pz', 'AB.B.theta.p.P4', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.b.FP2', 'AB.C.alpha.c.F7', 'AB.C.alpha.d.F3', 'AB.C.alpha.e.Fz', 'AB.C.alpha.f.F4', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.b.FP2', 'AB.D.beta.c.F7', 'AB.D.beta.d.F3', 'AB.D.beta.e.Fz', 'AB.D.beta.f.F4', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.k.C4', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.p.P4', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.e.Fz', 'AB.E.highbeta.f.F4', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.p.P4', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.m.T5', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.d.F3.s.O2', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.m.T5', 'COH.B.theta.e.Fz.n.P3', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.p.P4', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.p.P4', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.q.T6', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.i.C3.s.O2', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.e.Fz', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.h.T3', 'COH.C.alpha.b.FP2.i.C3', 'COH.C.alpha.b.FP2.j.Cz', 'COH.C.alpha.b.FP2.k.C4', 'COH.C.alpha.b.FP2.l.T4', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.n.P3', 'COH.C.alpha.b.FP2.o.Pz', 'COH.C.alpha.b.FP2.p.P4', 'COH.C.alpha.b.FP2.q.T6', 'COH.C.alpha.b.FP2.r.O1', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.e.Fz', 'COH.C.alpha.c.F7.f.F4', 'COH.C.alpha.c.F7.g.F8', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.k.C4', 'COH.C.alpha.c.F7.l.T4', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.o.Pz', 'COH.C.alpha.c.F7.p.P4', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.c.F7.s.O2', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.g.F8', 'COH.C.alpha.d.F3.h.T3', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.d.F3.k.C4', 'COH.C.alpha.d.F3.l.T4', 'COH.C.alpha.d.F3.m.T5', 'COH.C.alpha.d.F3.n.P3', 'COH.C.alpha.d.F3.o.Pz', 'COH.C.alpha.d.F3.p.P4', 'COH.C.alpha.d.F3.q.T6', 'COH.C.alpha.d.F3.r.O1', 'COH.C.alpha.d.F3.s.O2', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.e.Fz.g.F8', 'COH.C.alpha.e.Fz.h.T3', 'COH.C.alpha.e.Fz.i.C3', 'COH.C.alpha.e.Fz.j.Cz', 'COH.C.alpha.e.Fz.k.C4', 'COH.C.alpha.e.Fz.l.T4', 'COH.C.alpha.e.Fz.m.T5', 'COH.C.alpha.e.Fz.n.P3', 'COH.C.alpha.e.Fz.o.Pz', 'COH.C.alpha.e.Fz.p.P4', 'COH.C.alpha.e.Fz.q.T6', 'COH.C.alpha.e.Fz.r.O1', 'COH.C.alpha.e.Fz.s.O2', 'COH.C.alpha.f.F4.g.F8', 'COH.C.alpha.f.F4.h.T3', 'COH.C.alpha.f.F4.i.C3', 'COH.C.alpha.f.F4.j.Cz', 'COH.C.alpha.f.F4.k.C4', 'COH.C.alpha.f.F4.l.T4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.n.P3', 'COH.C.alpha.f.F4.o.Pz', 'COH.C.alpha.f.F4.p.P4', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.h.T3', 'COH.C.alpha.g.F8.i.C3', 'COH.C.alpha.g.F8.j.Cz', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.o.Pz', 'COH.C.alpha.g.F8.p.P4', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.j.Cz', 'COH.C.alpha.h.T3.k.C4', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.l.T4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.p.P4', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.l.T4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.p.P4', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.p.P4', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.l.T4', 'COH.D.beta.d.F3.m.T5', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.d.F3.p.P4', 'COH.D.beta.d.F3.q.T6', 'COH.D.beta.d.F3.r.O1', 'COH.D.beta.d.F3.s.O2', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.e.Fz.l.T4', 'COH.D.beta.e.Fz.m.T5', 'COH.D.beta.e.Fz.n.P3', 'COH.D.beta.e.Fz.o.Pz', 'COH.D.beta.e.Fz.p.P4', 'COH.D.beta.e.Fz.q.T6', 'COH.D.beta.e.Fz.r.O1', 'COH.D.beta.e.Fz.s.O2', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.o.Pz', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.m.T5', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.g.F8.r.O1', 'COH.D.beta.g.F8.s.O2', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.q.T6', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.o.Pz', 'COH.E.highbeta.b.FP2.p.P4', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.b.FP2.s.O2', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.o.Pz', 'COH.E.highbeta.c.F7.p.P4', 'COH.E.highbeta.c.F7.q.T6', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.c.F7.s.O2', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.l.T4', 'COH.E.highbeta.d.F3.m.T5', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.d.F3.o.Pz', 'COH.E.highbeta.d.F3.p.P4', 'COH.E.highbeta.d.F3.q.T6', 'COH.E.highbeta.d.F3.r.O1', 'COH.E.highbeta.d.F3.s.O2', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.n.P3', 'COH.E.highbeta.e.Fz.o.Pz', 'COH.E.highbeta.e.Fz.p.P4', 'COH.E.highbeta.e.Fz.q.T6', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.e.Fz.s.O2', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.l.T4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.o.Pz', 'COH.E.highbeta.f.F4.p.P4', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.r.O1', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.p.P4', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.k.C4', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.q.T6', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.i.C3.s.O2', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.l.T4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.o.Pz', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.b.FP2', 'COH.F.gamma.a.FP1.c.F7', 'COH.F.gamma.a.FP1.d.F3', 'COH.F.gamma.a.FP1.e.Fz', 'COH.F.gamma.a.FP1.f.F4', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.j.Cz', 'COH.F.gamma.a.FP1.k.C4', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.o.Pz', 'COH.F.gamma.a.FP1.p.P4', 'COH.F.gamma.a.FP1.q.T6', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.d.F3', 'COH.F.gamma.b.FP2.e.Fz', 'COH.F.gamma.b.FP2.f.F4', 'COH.F.gamma.b.FP2.g.F8', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.j.Cz', 'COH.F.gamma.b.FP2.k.C4', 'COH.F.gamma.b.FP2.l.T4', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.b.FP2.n.P3', 'COH.F.gamma.b.FP2.o.Pz', 'COH.F.gamma.b.FP2.p.P4', 'COH.F.gamma.b.FP2.q.T6', 'COH.F.gamma.b.FP2.r.O1', 'COH.F.gamma.b.FP2.s.O2', 'COH.F.gamma.c.F7.d.F3', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.f.F4', 'COH.F.gamma.c.F7.g.F8', 'COH.F.gamma.c.F7.h.T3', 'COH.F.gamma.c.F7.i.C3', 'COH.F.gamma.c.F7.j.Cz', 'COH.F.gamma.c.F7.k.C4', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.c.F7.m.T5', 'COH.F.gamma.c.F7.n.P3', 'COH.F.gamma.c.F7.o.Pz', 'COH.F.gamma.c.F7.p.P4', 'COH.F.gamma.c.F7.q.T6', 'COH.F.gamma.c.F7.r.O1', 'COH.F.gamma.c.F7.s.O2', 'COH.F.gamma.d.F3.e.Fz', 'COH.F.gamma.d.F3.f.F4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.d.F3.h.T3', 'COH.F.gamma.d.F3.i.C3', 'COH.F.gamma.d.F3.j.Cz', 'COH.F.gamma.d.F3.k.C4', 'COH.F.gamma.d.F3.l.T4', 'COH.F.gamma.d.F3.m.T5', 'COH.F.gamma.d.F3.n.P3', 'COH.F.gamma.d.F3.o.Pz', 'COH.F.gamma.d.F3.p.P4', 'COH.F.gamma.d.F3.q.T6', 'COH.F.gamma.d.F3.r.O1', 'COH.F.gamma.d.F3.s.O2', 'COH.F.gamma.e.Fz.f.F4', 'COH.F.gamma.e.Fz.g.F8', 'COH.F.gamma.e.Fz.h.T3', 'COH.F.gamma.e.Fz.i.C3', 'COH.F.gamma.e.Fz.j.Cz', 'COH.F.gamma.e.Fz.k.C4', 'COH.F.gamma.e.Fz.l.T4', 'COH.F.gamma.e.Fz.m.T5', 'COH.F.gamma.e.Fz.n.P3', 'COH.F.gamma.e.Fz.o.Pz', 'COH.F.gamma.e.Fz.p.P4', 'COH.F.gamma.e.Fz.q.T6', 'COH.F.gamma.e.Fz.r.O1', 'COH.F.gamma.e.Fz.s.O2', 'COH.F.gamma.f.F4.g.F8', 'COH.F.gamma.f.F4.h.T3', 'COH.F.gamma.f.F4.i.C3', 'COH.F.gamma.f.F4.j.Cz', 'COH.F.gamma.f.F4.k.C4', 'COH.F.gamma.f.F4.l.T4', 'COH.F.gamma.f.F4.m.T5', 'COH.F.gamma.f.F4.n.P3', 'COH.F.gamma.f.F4.o.Pz', 'COH.F.gamma.f.F4.p.P4', 'COH.F.gamma.f.F4.q.T6', 'COH.F.gamma.f.F4.r.O1', 'COH.F.gamma.f.F4.s.O2', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.i.C3', 'COH.F.gamma.g.F8.j.Cz', 'COH.F.gamma.g.F8.k.C4', 'COH.F.gamma.g.F8.l.T4', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.o.Pz', 'COH.F.gamma.g.F8.p.P4', 'COH.F.gamma.g.F8.q.T6', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.i.C3', 'COH.F.gamma.h.T3.j.Cz', 'COH.F.gamma.h.T3.k.C4', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.m.T5', 'COH.F.gamma.h.T3.n.P3', 'COH.F.gamma.h.T3.o.Pz', 'COH.F.gamma.h.T3.p.P4', 'COH.F.gamma.h.T3.q.T6', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.j.Cz', 'COH.F.gamma.i.C3.k.C4', 'COH.F.gamma.i.C3.l.T4', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.n.P3', 'COH.F.gamma.i.C3.o.Pz', 'COH.F.gamma.i.C3.p.P4', 'COH.F.gamma.i.C3.q.T6', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.i.C3.s.O2', 'COH.F.gamma.j.Cz.k.C4', 'COH.F.gamma.j.Cz.l.T4', 'COH.F.gamma.j.Cz.m.T5', 'COH.F.gamma.j.Cz.n.P3', 'COH.F.gamma.j.Cz.o.Pz', 'COH.F.gamma.j.Cz.p.P4', 'COH.F.gamma.j.Cz.q.T6', 'COH.F.gamma.j.Cz.r.O1', 'COH.F.gamma.j.Cz.s.O2', 'COH.F.gamma.k.C4.l.T4', 'COH.F.gamma.k.C4.m.T5', 'COH.F.gamma.k.C4.n.P3', 'COH.F.gamma.k.C4.o.Pz', 'COH.F.gamma.k.C4.p.P4', 'COH.F.gamma.k.C4.q.T6', 'COH.F.gamma.k.C4.r.O1', 'COH.F.gamma.k.C4.s.O2', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.n.P3', 'COH.F.gamma.l.T4.o.Pz', 'COH.F.gamma.l.T4.p.P4', 'COH.F.gamma.l.T4.q.T6', 'COH.F.gamma.l.T4.r.O1', 'COH.F.gamma.l.T4.s.O2', 'COH.F.gamma.m.T5.n.P3', 'COH.F.gamma.m.T5.o.Pz', 'COH.F.gamma.m.T5.p.P4', 'COH.F.gamma.m.T5.q.T6', 'COH.F.gamma.m.T5.r.O1', 'COH.F.gamma.m.T5.s.O2', 'COH.F.gamma.n.P3.o.Pz', 'COH.F.gamma.n.P3.p.P4', 'COH.F.gamma.n.P3.q.T6', 'COH.F.gamma.n.P3.r.O1', 'COH.F.gamma.n.P3.s.O2', 'COH.F.gamma.o.Pz.p.P4', 'COH.F.gamma.o.Pz.q.T6', 'COH.F.gamma.o.Pz.r.O1', 'COH.F.gamma.o.Pz.s.O2', 'COH.F.gamma.p.P4.q.T6', 'COH.F.gamma.p.P4.r.O1', 'COH.F.gamma.p.P4.s.O2', 'COH.F.gamma.q.T6.r.O1', 'COH.F.gamma.q.T6.s.O2', 'COH.F.gamma.r.O1.s.O2']
for i in arr3:
     fig=sm.qqplot(numeric_data1[i],line='45',xlabel="Theoretical "+i,ylabel=i+" feature value in the dataset") 
     plt.show()

### Label encoding

In [ ]:
enc = LabelEncoder()
# Iterate over each column in the categorical data in order to convert it to numeric data
for i in categorical_data.columns:
    enc.fit(categorical_data[i])
    categorical_data[i] = enc.transform(categorical_data[i])
    print(categorical_data[i]) 

X

y

In [ ]:
print(df8.columns.tolist())

### Correlation

In [ ]:
df8

In [ ]:
# Finding correlation as a part of inferential statistics using the corr() in the form of correlation matrix

corr = df8.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)           # Plotting the correlation matrix through heatmap
fig.colorbar(cax)
ticks = np.arange(0,len(df8.columns),1)
plt.xticks(rotation=90)
ax.set_xticklabels(df8.columns)
ax.set_yticklabels(df8.columns)
plt.show()

In [ ]:
corrMatrix=df8.corr().abs()

upperMatrix = corrMatrix.where(np.triu(np.ones(corrMatrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
corrFutures = [column for column in upperMatrix.columns if any(upperMatrix[column] > 0.9)]

df8.drop(columns=corrFutures, inplace = True)   # Dropping columns having correlation more than 0.90

In [ ]:
df8

In [ ]:
correlation = df8.corr()      # Plotting heatmap after dropping highly correlated columns
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlation,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(df8.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(df8.columns)
ax.set_yticklabels(df8.columns)
plt.show()

In [ ]:
print(df8.columns.tolist())

In [ ]:
df8

### Standardisation done for PCA and K-means clustering

In [ ]:
# Storing values of dataframe in an array
df8_feat = df8[['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.c.F7', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.c.F7', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.r.O1']]


In [ ]:
df8_feat.shape   # Displaying the size of df8_feat

In [ ]:
# Standardisation

from sklearn.preprocessing import StandardScaler
##Standardising the numerical data in order to get a normal scale 
df_st =  StandardScaler().fit_transform(df8_feat)  
#Storing the normalised data in another dataframe

df_n = pd.DataFrame(df_st,columns = ['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.c.F7', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.c.F7', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.r.O1'])
                    
df_n.head()                

### Normalisation

In [ ]:
df8.shape

In [ ]:
# Creating a variable X that contains all the features
X = df8.iloc[:, df8.columns != 'disorder'].values

# Creating a variable y that contains the target
y = df8.disorder.values

# Using MinMaxScaler in order to Normalise and Standarise the values of the X
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

### PCA

In [ ]:
from sklearn.decomposition import PCA                        # Importing the required libraries for Principal component analysis
from sklearn.decomposition import PCA as RandomizedPCA
import seaborn as sns

In [ ]:
##Applying PCA to find PCA's components 
pca_n = PCA().fit(df_n)
components = pca_n.transform(df_n)

In [ ]:
##Calculating the component variance 
pca_n.explained_variance_ratio_

In [ ]:
## Caluclation of cumulative proportion of variance
np.cumsum(pca_n.explained_variance_ratio_)

In [ ]:
##Loadings of each PCA components 
loadings = pd.DataFrame(pca_n.components_.T,
columns=['PC%s' % _ for _ in range(len(df_n.columns))],
index=df_n.columns)
print(loadings)

In [ ]:
##Plotting Cumalative values of PCs and adding a 85% threshold line.
plt.plot(np.cumsum(pca_n.explained_variance_ratio_), marker='o', linestyle='--', color='r',)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

plt.xlim(0,529)
plt.hlines(y=0.85, xmin=0, xmax=68)
plt.vlines(x=68, ymin=0, ymax=0.85)

In [ ]:
# Displaying the variances carried by different PCs
import matplotlib.pyplot as plt
pd.DataFrame(pca_n.explained_variance_ratio_).plot.bar(width=4)
plt.legend('')
plt.xlabel('Principal Components')
plt.ylabel('Explained Varience');
plt.rcParams["figure.figsize"] = (10,5)

In [ ]:
##Heatmap of PCs (Principal components)

ax = sns.heatmap(loadings, cmap="PiYG")
plt.title('Heatmap of PCs')
plt.show()


In [ ]:
##Screeplot of PCs
PC_values = np.arange(pca_n.n_components_) + 1
plt.plot(PC_values, pca_n.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

### K-means clustering

In [ ]:
from sklearn.cluster import KMeans            # Importing the required libraries for k-means clustering
from sklearn.metrics import silhouette_score
import seaborn as sb    # for seeing visually how the data distribution

In [ ]:
#Using For Loop to get the Silhoette Score at different values of n_clusters
n = [2,3,4,5,6]
silhouette_sco = []
max_score = 0
max_n = 0
for i in n:
    kmeans = KMeans(n_clusters = i, random_state = 45)
    kmeans.fit(df_n)
    y_kmeans = kmeans.predict(df_n)
    val = silhouette_score(df_n, y_kmeans)
    silhouette_sco.append(val)  

In [ ]:
#Plotting the n_clusters value against Silhoette Score
fig=plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(n,silhouette_sco)
plt.title('Comparison of Number of Clusters on the basis of Silhoette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhoette Score')
plt.show()

In [ ]:
#Using KMeans for Number of Clusters value =2

arr=['AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1', 'AB.B.theta.c.F7', 'AB.B.theta.d.F3', 'AB.B.theta.e.Fz', 'AB.B.theta.g.F8', 'AB.B.theta.h.T3', 'AB.B.theta.i.C3', 'AB.B.theta.j.Cz', 'AB.B.theta.k.C4', 'AB.B.theta.l.T4', 'AB.B.theta.m.T5', 'AB.B.theta.n.P3', 'AB.B.theta.q.T6', 'AB.B.theta.r.O1', 'AB.B.theta.s.O2', 'AB.C.alpha.a.FP1', 'AB.C.alpha.c.F7', 'AB.C.alpha.g.F8', 'AB.C.alpha.h.T3', 'AB.C.alpha.i.C3', 'AB.C.alpha.j.Cz', 'AB.C.alpha.k.C4', 'AB.C.alpha.l.T4', 'AB.C.alpha.m.T5', 'AB.C.alpha.n.P3', 'AB.C.alpha.o.Pz', 'AB.C.alpha.p.P4', 'AB.C.alpha.q.T6', 'AB.C.alpha.r.O1', 'AB.C.alpha.s.O2', 'AB.D.beta.a.FP1', 'AB.D.beta.c.F7', 'AB.D.beta.g.F8', 'AB.D.beta.h.T3', 'AB.D.beta.i.C3', 'AB.D.beta.j.Cz', 'AB.D.beta.l.T4', 'AB.D.beta.m.T5', 'AB.D.beta.n.P3', 'AB.D.beta.o.Pz', 'AB.D.beta.q.T6', 'AB.D.beta.r.O1', 'AB.D.beta.s.O2', 'AB.E.highbeta.a.FP1', 'AB.E.highbeta.b.FP2', 'AB.E.highbeta.c.F7', 'AB.E.highbeta.d.F3', 'AB.E.highbeta.g.F8', 'AB.E.highbeta.h.T3', 'AB.E.highbeta.i.C3', 'AB.E.highbeta.j.Cz', 'AB.E.highbeta.k.C4', 'AB.E.highbeta.l.T4', 'AB.E.highbeta.m.T5', 'AB.E.highbeta.n.P3', 'AB.E.highbeta.o.Pz', 'AB.E.highbeta.p.P4', 'AB.E.highbeta.q.T6', 'AB.E.highbeta.r.O1', 'AB.E.highbeta.s.O2', 'AB.F.gamma.a.FP1', 'AB.F.gamma.b.FP2', 'AB.F.gamma.c.F7', 'AB.F.gamma.d.F3', 'AB.F.gamma.e.Fz', 'AB.F.gamma.f.F4', 'AB.F.gamma.g.F8', 'AB.F.gamma.h.T3', 'AB.F.gamma.i.C3', 'AB.F.gamma.j.Cz', 'AB.F.gamma.k.C4', 'AB.F.gamma.l.T4', 'AB.F.gamma.m.T5', 'AB.F.gamma.n.P3', 'AB.F.gamma.o.Pz', 'AB.F.gamma.p.P4', 'AB.F.gamma.q.T6', 'AB.F.gamma.r.O1', 'AB.F.gamma.s.O2', 'COH.A.delta.a.FP1.b.FP2', 'COH.A.delta.a.FP1.c.F7', 'COH.A.delta.a.FP1.d.F3', 'COH.A.delta.a.FP1.e.Fz', 'COH.A.delta.a.FP1.f.F4', 'COH.A.delta.a.FP1.g.F8', 'COH.A.delta.a.FP1.h.T3', 'COH.A.delta.a.FP1.i.C3', 'COH.A.delta.a.FP1.j.Cz', 'COH.A.delta.a.FP1.k.C4', 'COH.A.delta.a.FP1.l.T4', 'COH.A.delta.a.FP1.m.T5', 'COH.A.delta.a.FP1.n.P3', 'COH.A.delta.a.FP1.o.Pz', 'COH.A.delta.a.FP1.p.P4', 'COH.A.delta.a.FP1.q.T6', 'COH.A.delta.a.FP1.r.O1', 'COH.A.delta.a.FP1.s.O2', 'COH.A.delta.b.FP2.c.F7', 'COH.A.delta.b.FP2.d.F3', 'COH.A.delta.b.FP2.e.Fz', 'COH.A.delta.b.FP2.f.F4', 'COH.A.delta.b.FP2.g.F8', 'COH.A.delta.b.FP2.h.T3', 'COH.A.delta.b.FP2.i.C3', 'COH.A.delta.b.FP2.j.Cz', 'COH.A.delta.b.FP2.k.C4', 'COH.A.delta.b.FP2.l.T4', 'COH.A.delta.b.FP2.m.T5', 'COH.A.delta.b.FP2.n.P3', 'COH.A.delta.b.FP2.o.Pz', 'COH.A.delta.b.FP2.p.P4', 'COH.A.delta.b.FP2.q.T6', 'COH.A.delta.b.FP2.r.O1', 'COH.A.delta.b.FP2.s.O2', 'COH.A.delta.c.F7.d.F3', 'COH.A.delta.c.F7.e.Fz', 'COH.A.delta.c.F7.f.F4', 'COH.A.delta.c.F7.g.F8', 'COH.A.delta.c.F7.h.T3', 'COH.A.delta.c.F7.i.C3', 'COH.A.delta.c.F7.j.Cz', 'COH.A.delta.c.F7.k.C4', 'COH.A.delta.c.F7.l.T4', 'COH.A.delta.c.F7.m.T5', 'COH.A.delta.c.F7.n.P3', 'COH.A.delta.c.F7.o.Pz', 'COH.A.delta.c.F7.p.P4', 'COH.A.delta.c.F7.q.T6', 'COH.A.delta.c.F7.r.O1', 'COH.A.delta.c.F7.s.O2', 'COH.A.delta.d.F3.e.Fz', 'COH.A.delta.d.F3.f.F4', 'COH.A.delta.d.F3.g.F8', 'COH.A.delta.d.F3.h.T3', 'COH.A.delta.d.F3.i.C3', 'COH.A.delta.d.F3.j.Cz', 'COH.A.delta.d.F3.k.C4', 'COH.A.delta.d.F3.l.T4', 'COH.A.delta.d.F3.m.T5', 'COH.A.delta.d.F3.n.P3', 'COH.A.delta.d.F3.o.Pz', 'COH.A.delta.d.F3.p.P4', 'COH.A.delta.d.F3.q.T6', 'COH.A.delta.d.F3.r.O1', 'COH.A.delta.d.F3.s.O2', 'COH.A.delta.e.Fz.f.F4', 'COH.A.delta.e.Fz.g.F8', 'COH.A.delta.e.Fz.h.T3', 'COH.A.delta.e.Fz.i.C3', 'COH.A.delta.e.Fz.j.Cz', 'COH.A.delta.e.Fz.k.C4', 'COH.A.delta.e.Fz.l.T4', 'COH.A.delta.e.Fz.m.T5', 'COH.A.delta.e.Fz.n.P3', 'COH.A.delta.e.Fz.o.Pz', 'COH.A.delta.e.Fz.p.P4', 'COH.A.delta.e.Fz.q.T6', 'COH.A.delta.e.Fz.r.O1', 'COH.A.delta.e.Fz.s.O2', 'COH.A.delta.f.F4.g.F8', 'COH.A.delta.f.F4.h.T3', 'COH.A.delta.f.F4.i.C3', 'COH.A.delta.f.F4.j.Cz', 'COH.A.delta.f.F4.k.C4', 'COH.A.delta.f.F4.l.T4', 'COH.A.delta.f.F4.m.T5', 'COH.A.delta.f.F4.n.P3', 'COH.A.delta.f.F4.o.Pz', 'COH.A.delta.f.F4.q.T6', 'COH.A.delta.f.F4.r.O1', 'COH.A.delta.f.F4.s.O2', 'COH.A.delta.g.F8.h.T3', 'COH.A.delta.g.F8.i.C3', 'COH.A.delta.g.F8.j.Cz', 'COH.A.delta.g.F8.k.C4', 'COH.A.delta.g.F8.l.T4', 'COH.A.delta.g.F8.m.T5', 'COH.A.delta.g.F8.n.P3', 'COH.A.delta.g.F8.o.Pz', 'COH.A.delta.g.F8.p.P4', 'COH.A.delta.g.F8.q.T6', 'COH.A.delta.g.F8.r.O1', 'COH.A.delta.g.F8.s.O2', 'COH.A.delta.h.T3.i.C3', 'COH.A.delta.h.T3.j.Cz', 'COH.A.delta.h.T3.k.C4', 'COH.A.delta.h.T3.l.T4', 'COH.A.delta.h.T3.m.T5', 'COH.A.delta.h.T3.n.P3', 'COH.A.delta.h.T3.o.Pz', 'COH.A.delta.h.T3.p.P4', 'COH.A.delta.h.T3.q.T6', 'COH.A.delta.h.T3.r.O1', 'COH.A.delta.h.T3.s.O2', 'COH.A.delta.i.C3.j.Cz', 'COH.A.delta.i.C3.k.C4', 'COH.A.delta.i.C3.l.T4', 'COH.A.delta.i.C3.m.T5', 'COH.A.delta.i.C3.n.P3', 'COH.A.delta.i.C3.o.Pz', 'COH.A.delta.i.C3.p.P4', 'COH.A.delta.i.C3.q.T6', 'COH.A.delta.i.C3.r.O1', 'COH.A.delta.i.C3.s.O2', 'COH.A.delta.j.Cz.k.C4', 'COH.A.delta.j.Cz.l.T4', 'COH.A.delta.j.Cz.m.T5', 'COH.A.delta.j.Cz.n.P3', 'COH.A.delta.j.Cz.o.Pz', 'COH.A.delta.j.Cz.p.P4', 'COH.A.delta.j.Cz.q.T6', 'COH.A.delta.j.Cz.r.O1', 'COH.A.delta.j.Cz.s.O2', 'COH.A.delta.k.C4.l.T4', 'COH.A.delta.k.C4.m.T5', 'COH.A.delta.k.C4.n.P3', 'COH.A.delta.k.C4.o.Pz', 'COH.A.delta.k.C4.p.P4', 'COH.A.delta.k.C4.q.T6', 'COH.A.delta.k.C4.r.O1', 'COH.A.delta.k.C4.s.O2', 'COH.A.delta.l.T4.m.T5', 'COH.A.delta.l.T4.n.P3', 'COH.A.delta.l.T4.o.Pz', 'COH.A.delta.l.T4.p.P4', 'COH.A.delta.l.T4.q.T6', 'COH.A.delta.l.T4.r.O1', 'COH.A.delta.l.T4.s.O2', 'COH.A.delta.m.T5.n.P3', 'COH.A.delta.m.T5.o.Pz', 'COH.A.delta.m.T5.p.P4', 'COH.A.delta.m.T5.q.T6', 'COH.A.delta.m.T5.r.O1', 'COH.A.delta.m.T5.s.O2', 'COH.A.delta.n.P3.o.Pz', 'COH.A.delta.n.P3.p.P4', 'COH.A.delta.n.P3.q.T6', 'COH.A.delta.n.P3.r.O1', 'COH.A.delta.n.P3.s.O2', 'COH.A.delta.o.Pz.p.P4', 'COH.A.delta.o.Pz.q.T6', 'COH.A.delta.o.Pz.r.O1', 'COH.A.delta.o.Pz.s.O2', 'COH.A.delta.p.P4.q.T6', 'COH.A.delta.p.P4.r.O1', 'COH.A.delta.p.P4.s.O2', 'COH.A.delta.q.T6.r.O1', 'COH.A.delta.q.T6.s.O2', 'COH.A.delta.r.O1.s.O2', 'COH.B.theta.a.FP1.b.FP2', 'COH.B.theta.a.FP1.c.F7', 'COH.B.theta.a.FP1.d.F3', 'COH.B.theta.a.FP1.e.Fz', 'COH.B.theta.a.FP1.f.F4', 'COH.B.theta.a.FP1.g.F8', 'COH.B.theta.a.FP1.h.T3', 'COH.B.theta.a.FP1.i.C3', 'COH.B.theta.a.FP1.j.Cz', 'COH.B.theta.a.FP1.k.C4', 'COH.B.theta.a.FP1.l.T4', 'COH.B.theta.a.FP1.m.T5', 'COH.B.theta.a.FP1.n.P3', 'COH.B.theta.a.FP1.o.Pz', 'COH.B.theta.a.FP1.p.P4', 'COH.B.theta.a.FP1.q.T6', 'COH.B.theta.a.FP1.r.O1', 'COH.B.theta.a.FP1.s.O2', 'COH.B.theta.b.FP2.c.F7', 'COH.B.theta.b.FP2.d.F3', 'COH.B.theta.b.FP2.e.Fz', 'COH.B.theta.b.FP2.f.F4', 'COH.B.theta.b.FP2.g.F8', 'COH.B.theta.b.FP2.h.T3', 'COH.B.theta.b.FP2.i.C3', 'COH.B.theta.b.FP2.j.Cz', 'COH.B.theta.b.FP2.k.C4', 'COH.B.theta.b.FP2.l.T4', 'COH.B.theta.b.FP2.m.T5', 'COH.B.theta.b.FP2.n.P3', 'COH.B.theta.b.FP2.o.Pz', 'COH.B.theta.b.FP2.p.P4', 'COH.B.theta.b.FP2.q.T6', 'COH.B.theta.b.FP2.r.O1', 'COH.B.theta.b.FP2.s.O2', 'COH.B.theta.c.F7.d.F3', 'COH.B.theta.c.F7.e.Fz', 'COH.B.theta.c.F7.f.F4', 'COH.B.theta.c.F7.g.F8', 'COH.B.theta.c.F7.h.T3', 'COH.B.theta.c.F7.i.C3', 'COH.B.theta.c.F7.j.Cz', 'COH.B.theta.c.F7.k.C4', 'COH.B.theta.c.F7.l.T4', 'COH.B.theta.c.F7.m.T5', 'COH.B.theta.c.F7.n.P3', 'COH.B.theta.c.F7.o.Pz', 'COH.B.theta.c.F7.p.P4', 'COH.B.theta.c.F7.q.T6', 'COH.B.theta.c.F7.r.O1', 'COH.B.theta.c.F7.s.O2', 'COH.B.theta.d.F3.e.Fz', 'COH.B.theta.d.F3.f.F4', 'COH.B.theta.d.F3.g.F8', 'COH.B.theta.d.F3.h.T3', 'COH.B.theta.d.F3.i.C3', 'COH.B.theta.d.F3.j.Cz', 'COH.B.theta.d.F3.k.C4', 'COH.B.theta.d.F3.l.T4', 'COH.B.theta.d.F3.n.P3', 'COH.B.theta.d.F3.o.Pz', 'COH.B.theta.d.F3.p.P4', 'COH.B.theta.d.F3.q.T6', 'COH.B.theta.d.F3.r.O1', 'COH.B.theta.e.Fz.f.F4', 'COH.B.theta.e.Fz.g.F8', 'COH.B.theta.e.Fz.h.T3', 'COH.B.theta.e.Fz.i.C3', 'COH.B.theta.e.Fz.j.Cz', 'COH.B.theta.e.Fz.k.C4', 'COH.B.theta.e.Fz.l.T4', 'COH.B.theta.e.Fz.o.Pz', 'COH.B.theta.e.Fz.q.T6', 'COH.B.theta.e.Fz.r.O1', 'COH.B.theta.e.Fz.s.O2', 'COH.B.theta.f.F4.g.F8', 'COH.B.theta.f.F4.h.T3', 'COH.B.theta.f.F4.i.C3', 'COH.B.theta.f.F4.j.Cz', 'COH.B.theta.f.F4.k.C4', 'COH.B.theta.f.F4.l.T4', 'COH.B.theta.f.F4.m.T5', 'COH.B.theta.f.F4.n.P3', 'COH.B.theta.f.F4.o.Pz', 'COH.B.theta.f.F4.q.T6', 'COH.B.theta.f.F4.r.O1', 'COH.B.theta.f.F4.s.O2', 'COH.B.theta.g.F8.h.T3', 'COH.B.theta.g.F8.i.C3', 'COH.B.theta.g.F8.j.Cz', 'COH.B.theta.g.F8.k.C4', 'COH.B.theta.g.F8.l.T4', 'COH.B.theta.g.F8.m.T5', 'COH.B.theta.g.F8.n.P3', 'COH.B.theta.g.F8.o.Pz', 'COH.B.theta.g.F8.p.P4', 'COH.B.theta.g.F8.q.T6', 'COH.B.theta.g.F8.r.O1', 'COH.B.theta.g.F8.s.O2', 'COH.B.theta.h.T3.i.C3', 'COH.B.theta.h.T3.j.Cz', 'COH.B.theta.h.T3.k.C4', 'COH.B.theta.h.T3.l.T4', 'COH.B.theta.h.T3.m.T5', 'COH.B.theta.h.T3.n.P3', 'COH.B.theta.h.T3.o.Pz', 'COH.B.theta.h.T3.p.P4', 'COH.B.theta.h.T3.q.T6', 'COH.B.theta.h.T3.r.O1', 'COH.B.theta.h.T3.s.O2', 'COH.B.theta.i.C3.j.Cz', 'COH.B.theta.i.C3.k.C4', 'COH.B.theta.i.C3.l.T4', 'COH.B.theta.i.C3.m.T5', 'COH.B.theta.i.C3.n.P3', 'COH.B.theta.i.C3.o.Pz', 'COH.B.theta.i.C3.p.P4', 'COH.B.theta.i.C3.r.O1', 'COH.B.theta.j.Cz.k.C4', 'COH.B.theta.j.Cz.l.T4', 'COH.B.theta.j.Cz.m.T5', 'COH.B.theta.j.Cz.n.P3', 'COH.B.theta.j.Cz.o.Pz', 'COH.B.theta.j.Cz.p.P4', 'COH.B.theta.j.Cz.q.T6', 'COH.B.theta.j.Cz.r.O1', 'COH.B.theta.j.Cz.s.O2', 'COH.B.theta.k.C4.l.T4', 'COH.B.theta.k.C4.m.T5', 'COH.B.theta.k.C4.n.P3', 'COH.B.theta.k.C4.o.Pz', 'COH.B.theta.k.C4.p.P4', 'COH.B.theta.k.C4.q.T6', 'COH.B.theta.k.C4.r.O1', 'COH.B.theta.k.C4.s.O2', 'COH.B.theta.l.T4.m.T5', 'COH.B.theta.l.T4.n.P3', 'COH.B.theta.l.T4.o.Pz', 'COH.B.theta.l.T4.p.P4', 'COH.B.theta.l.T4.q.T6', 'COH.B.theta.l.T4.r.O1', 'COH.B.theta.l.T4.s.O2', 'COH.B.theta.m.T5.n.P3', 'COH.B.theta.m.T5.o.Pz', 'COH.B.theta.m.T5.p.P4', 'COH.B.theta.m.T5.q.T6', 'COH.B.theta.m.T5.r.O1', 'COH.B.theta.m.T5.s.O2', 'COH.B.theta.n.P3.o.Pz', 'COH.B.theta.n.P3.p.P4', 'COH.B.theta.n.P3.q.T6', 'COH.B.theta.n.P3.r.O1', 'COH.B.theta.n.P3.s.O2', 'COH.B.theta.o.Pz.p.P4', 'COH.B.theta.o.Pz.q.T6', 'COH.B.theta.o.Pz.r.O1', 'COH.B.theta.o.Pz.s.O2', 'COH.B.theta.p.P4.q.T6', 'COH.B.theta.p.P4.r.O1', 'COH.B.theta.p.P4.s.O2', 'COH.B.theta.q.T6.r.O1', 'COH.B.theta.q.T6.s.O2', 'COH.B.theta.r.O1.s.O2', 'COH.C.alpha.a.FP1.b.FP2', 'COH.C.alpha.a.FP1.c.F7', 'COH.C.alpha.a.FP1.d.F3', 'COH.C.alpha.a.FP1.e.Fz', 'COH.C.alpha.a.FP1.f.F4', 'COH.C.alpha.a.FP1.g.F8', 'COH.C.alpha.a.FP1.h.T3', 'COH.C.alpha.a.FP1.i.C3', 'COH.C.alpha.a.FP1.j.Cz', 'COH.C.alpha.a.FP1.k.C4', 'COH.C.alpha.a.FP1.l.T4', 'COH.C.alpha.a.FP1.m.T5', 'COH.C.alpha.a.FP1.n.P3', 'COH.C.alpha.a.FP1.o.Pz', 'COH.C.alpha.a.FP1.p.P4', 'COH.C.alpha.a.FP1.q.T6', 'COH.C.alpha.a.FP1.r.O1', 'COH.C.alpha.a.FP1.s.O2', 'COH.C.alpha.b.FP2.c.F7', 'COH.C.alpha.b.FP2.d.F3', 'COH.C.alpha.b.FP2.f.F4', 'COH.C.alpha.b.FP2.g.F8', 'COH.C.alpha.b.FP2.m.T5', 'COH.C.alpha.b.FP2.s.O2', 'COH.C.alpha.c.F7.d.F3', 'COH.C.alpha.c.F7.h.T3', 'COH.C.alpha.c.F7.i.C3', 'COH.C.alpha.c.F7.j.Cz', 'COH.C.alpha.c.F7.m.T5', 'COH.C.alpha.c.F7.n.P3', 'COH.C.alpha.c.F7.q.T6', 'COH.C.alpha.c.F7.r.O1', 'COH.C.alpha.d.F3.e.Fz', 'COH.C.alpha.d.F3.f.F4', 'COH.C.alpha.d.F3.i.C3', 'COH.C.alpha.d.F3.j.Cz', 'COH.C.alpha.e.Fz.f.F4', 'COH.C.alpha.f.F4.m.T5', 'COH.C.alpha.f.F4.q.T6', 'COH.C.alpha.f.F4.r.O1', 'COH.C.alpha.f.F4.s.O2', 'COH.C.alpha.g.F8.k.C4', 'COH.C.alpha.g.F8.l.T4', 'COH.C.alpha.g.F8.m.T5', 'COH.C.alpha.g.F8.n.P3', 'COH.C.alpha.g.F8.q.T6', 'COH.C.alpha.g.F8.r.O1', 'COH.C.alpha.g.F8.s.O2', 'COH.C.alpha.h.T3.i.C3', 'COH.C.alpha.h.T3.l.T4', 'COH.C.alpha.h.T3.m.T5', 'COH.C.alpha.h.T3.n.P3', 'COH.C.alpha.h.T3.o.Pz', 'COH.C.alpha.h.T3.p.P4', 'COH.C.alpha.h.T3.q.T6', 'COH.C.alpha.h.T3.r.O1', 'COH.C.alpha.h.T3.s.O2', 'COH.C.alpha.i.C3.j.Cz', 'COH.C.alpha.i.C3.k.C4', 'COH.C.alpha.i.C3.m.T5', 'COH.C.alpha.i.C3.n.P3', 'COH.C.alpha.i.C3.o.Pz', 'COH.C.alpha.i.C3.q.T6', 'COH.C.alpha.i.C3.r.O1', 'COH.C.alpha.i.C3.s.O2', 'COH.C.alpha.j.Cz.k.C4', 'COH.C.alpha.j.Cz.m.T5', 'COH.C.alpha.j.Cz.n.P3', 'COH.C.alpha.j.Cz.o.Pz', 'COH.C.alpha.j.Cz.q.T6', 'COH.C.alpha.j.Cz.r.O1', 'COH.C.alpha.j.Cz.s.O2', 'COH.C.alpha.k.C4.l.T4', 'COH.C.alpha.k.C4.m.T5', 'COH.C.alpha.k.C4.n.P3', 'COH.C.alpha.k.C4.o.Pz', 'COH.C.alpha.k.C4.p.P4', 'COH.C.alpha.k.C4.q.T6', 'COH.C.alpha.k.C4.r.O1', 'COH.C.alpha.k.C4.s.O2', 'COH.C.alpha.l.T4.m.T5', 'COH.C.alpha.l.T4.n.P3', 'COH.C.alpha.l.T4.o.Pz', 'COH.C.alpha.l.T4.p.P4', 'COH.C.alpha.l.T4.q.T6', 'COH.C.alpha.l.T4.r.O1', 'COH.C.alpha.l.T4.s.O2', 'COH.C.alpha.m.T5.n.P3', 'COH.C.alpha.m.T5.o.Pz', 'COH.C.alpha.m.T5.p.P4', 'COH.C.alpha.m.T5.q.T6', 'COH.C.alpha.m.T5.r.O1', 'COH.C.alpha.m.T5.s.O2', 'COH.C.alpha.n.P3.o.Pz', 'COH.C.alpha.n.P3.p.P4', 'COH.C.alpha.n.P3.q.T6', 'COH.C.alpha.n.P3.r.O1', 'COH.C.alpha.n.P3.s.O2', 'COH.C.alpha.o.Pz.p.P4', 'COH.C.alpha.o.Pz.q.T6', 'COH.C.alpha.o.Pz.r.O1', 'COH.C.alpha.o.Pz.s.O2', 'COH.C.alpha.p.P4.q.T6', 'COH.C.alpha.p.P4.r.O1', 'COH.C.alpha.p.P4.s.O2', 'COH.C.alpha.q.T6.r.O1', 'COH.C.alpha.q.T6.s.O2', 'COH.C.alpha.r.O1.s.O2', 'COH.D.beta.a.FP1.b.FP2', 'COH.D.beta.a.FP1.c.F7', 'COH.D.beta.a.FP1.d.F3', 'COH.D.beta.a.FP1.e.Fz', 'COH.D.beta.a.FP1.f.F4', 'COH.D.beta.a.FP1.g.F8', 'COH.D.beta.a.FP1.h.T3', 'COH.D.beta.a.FP1.i.C3', 'COH.D.beta.a.FP1.j.Cz', 'COH.D.beta.a.FP1.k.C4', 'COH.D.beta.a.FP1.l.T4', 'COH.D.beta.a.FP1.m.T5', 'COH.D.beta.a.FP1.n.P3', 'COH.D.beta.a.FP1.o.Pz', 'COH.D.beta.a.FP1.p.P4', 'COH.D.beta.a.FP1.q.T6', 'COH.D.beta.a.FP1.r.O1', 'COH.D.beta.a.FP1.s.O2', 'COH.D.beta.b.FP2.c.F7', 'COH.D.beta.b.FP2.d.F3', 'COH.D.beta.b.FP2.e.Fz', 'COH.D.beta.b.FP2.f.F4', 'COH.D.beta.b.FP2.g.F8', 'COH.D.beta.b.FP2.h.T3', 'COH.D.beta.b.FP2.i.C3', 'COH.D.beta.b.FP2.j.Cz', 'COH.D.beta.b.FP2.k.C4', 'COH.D.beta.b.FP2.l.T4', 'COH.D.beta.b.FP2.m.T5', 'COH.D.beta.b.FP2.n.P3', 'COH.D.beta.b.FP2.o.Pz', 'COH.D.beta.b.FP2.p.P4', 'COH.D.beta.b.FP2.q.T6', 'COH.D.beta.b.FP2.r.O1', 'COH.D.beta.b.FP2.s.O2', 'COH.D.beta.c.F7.d.F3', 'COH.D.beta.c.F7.e.Fz', 'COH.D.beta.c.F7.f.F4', 'COH.D.beta.c.F7.g.F8', 'COH.D.beta.c.F7.h.T3', 'COH.D.beta.c.F7.i.C3', 'COH.D.beta.c.F7.j.Cz', 'COH.D.beta.c.F7.k.C4', 'COH.D.beta.c.F7.l.T4', 'COH.D.beta.c.F7.m.T5', 'COH.D.beta.c.F7.n.P3', 'COH.D.beta.c.F7.o.Pz', 'COH.D.beta.c.F7.q.T6', 'COH.D.beta.c.F7.r.O1', 'COH.D.beta.c.F7.s.O2', 'COH.D.beta.d.F3.e.Fz', 'COH.D.beta.d.F3.f.F4', 'COH.D.beta.d.F3.g.F8', 'COH.D.beta.d.F3.h.T3', 'COH.D.beta.d.F3.i.C3', 'COH.D.beta.d.F3.j.Cz', 'COH.D.beta.d.F3.k.C4', 'COH.D.beta.d.F3.n.P3', 'COH.D.beta.d.F3.o.Pz', 'COH.D.beta.e.Fz.f.F4', 'COH.D.beta.e.Fz.g.F8', 'COH.D.beta.e.Fz.h.T3', 'COH.D.beta.e.Fz.i.C3', 'COH.D.beta.e.Fz.j.Cz', 'COH.D.beta.e.Fz.k.C4', 'COH.D.beta.f.F4.g.F8', 'COH.D.beta.f.F4.h.T3', 'COH.D.beta.f.F4.i.C3', 'COH.D.beta.f.F4.j.Cz', 'COH.D.beta.f.F4.k.C4', 'COH.D.beta.f.F4.l.T4', 'COH.D.beta.f.F4.m.T5', 'COH.D.beta.f.F4.n.P3', 'COH.D.beta.f.F4.p.P4', 'COH.D.beta.f.F4.q.T6', 'COH.D.beta.f.F4.r.O1', 'COH.D.beta.f.F4.s.O2', 'COH.D.beta.g.F8.h.T3', 'COH.D.beta.g.F8.i.C3', 'COH.D.beta.g.F8.j.Cz', 'COH.D.beta.g.F8.k.C4', 'COH.D.beta.g.F8.l.T4', 'COH.D.beta.g.F8.n.P3', 'COH.D.beta.g.F8.o.Pz', 'COH.D.beta.g.F8.p.P4', 'COH.D.beta.g.F8.q.T6', 'COH.D.beta.h.T3.i.C3', 'COH.D.beta.h.T3.j.Cz', 'COH.D.beta.h.T3.k.C4', 'COH.D.beta.h.T3.l.T4', 'COH.D.beta.h.T3.m.T5', 'COH.D.beta.h.T3.n.P3', 'COH.D.beta.h.T3.o.Pz', 'COH.D.beta.h.T3.p.P4', 'COH.D.beta.h.T3.q.T6', 'COH.D.beta.h.T3.r.O1', 'COH.D.beta.h.T3.s.O2', 'COH.D.beta.i.C3.j.Cz', 'COH.D.beta.i.C3.k.C4', 'COH.D.beta.i.C3.l.T4', 'COH.D.beta.i.C3.m.T5', 'COH.D.beta.i.C3.n.P3', 'COH.D.beta.i.C3.o.Pz', 'COH.D.beta.i.C3.p.P4', 'COH.D.beta.i.C3.r.O1', 'COH.D.beta.i.C3.s.O2', 'COH.D.beta.j.Cz.k.C4', 'COH.D.beta.j.Cz.l.T4', 'COH.D.beta.j.Cz.m.T5', 'COH.D.beta.j.Cz.n.P3', 'COH.D.beta.j.Cz.o.Pz', 'COH.D.beta.j.Cz.p.P4', 'COH.D.beta.j.Cz.q.T6', 'COH.D.beta.j.Cz.r.O1', 'COH.D.beta.j.Cz.s.O2', 'COH.D.beta.k.C4.l.T4', 'COH.D.beta.k.C4.m.T5', 'COH.D.beta.k.C4.n.P3', 'COH.D.beta.k.C4.o.Pz', 'COH.D.beta.k.C4.p.P4', 'COH.D.beta.k.C4.q.T6', 'COH.D.beta.k.C4.r.O1', 'COH.D.beta.k.C4.s.O2', 'COH.D.beta.l.T4.m.T5', 'COH.D.beta.l.T4.n.P3', 'COH.D.beta.l.T4.o.Pz', 'COH.D.beta.l.T4.p.P4', 'COH.D.beta.l.T4.q.T6', 'COH.D.beta.l.T4.r.O1', 'COH.D.beta.l.T4.s.O2', 'COH.D.beta.m.T5.n.P3', 'COH.D.beta.m.T5.o.Pz', 'COH.D.beta.m.T5.p.P4', 'COH.D.beta.m.T5.q.T6', 'COH.D.beta.m.T5.r.O1', 'COH.D.beta.m.T5.s.O2', 'COH.D.beta.n.P3.o.Pz', 'COH.D.beta.n.P3.p.P4', 'COH.D.beta.n.P3.q.T6', 'COH.D.beta.n.P3.r.O1', 'COH.D.beta.n.P3.s.O2', 'COH.D.beta.o.Pz.p.P4', 'COH.D.beta.o.Pz.q.T6', 'COH.D.beta.o.Pz.r.O1', 'COH.D.beta.o.Pz.s.O2', 'COH.D.beta.p.P4.q.T6', 'COH.D.beta.p.P4.r.O1', 'COH.D.beta.p.P4.s.O2', 'COH.D.beta.q.T6.r.O1', 'COH.D.beta.q.T6.s.O2', 'COH.D.beta.r.O1.s.O2', 'COH.E.highbeta.a.FP1.b.FP2', 'COH.E.highbeta.a.FP1.c.F7', 'COH.E.highbeta.a.FP1.d.F3', 'COH.E.highbeta.a.FP1.e.Fz', 'COH.E.highbeta.a.FP1.f.F4', 'COH.E.highbeta.a.FP1.g.F8', 'COH.E.highbeta.a.FP1.h.T3', 'COH.E.highbeta.a.FP1.i.C3', 'COH.E.highbeta.a.FP1.j.Cz', 'COH.E.highbeta.a.FP1.k.C4', 'COH.E.highbeta.a.FP1.l.T4', 'COH.E.highbeta.a.FP1.m.T5', 'COH.E.highbeta.a.FP1.n.P3', 'COH.E.highbeta.a.FP1.o.Pz', 'COH.E.highbeta.a.FP1.p.P4', 'COH.E.highbeta.a.FP1.q.T6', 'COH.E.highbeta.a.FP1.r.O1', 'COH.E.highbeta.a.FP1.s.O2', 'COH.E.highbeta.b.FP2.c.F7', 'COH.E.highbeta.b.FP2.d.F3', 'COH.E.highbeta.b.FP2.e.Fz', 'COH.E.highbeta.b.FP2.f.F4', 'COH.E.highbeta.b.FP2.g.F8', 'COH.E.highbeta.b.FP2.h.T3', 'COH.E.highbeta.b.FP2.i.C3', 'COH.E.highbeta.b.FP2.j.Cz', 'COH.E.highbeta.b.FP2.k.C4', 'COH.E.highbeta.b.FP2.l.T4', 'COH.E.highbeta.b.FP2.m.T5', 'COH.E.highbeta.b.FP2.n.P3', 'COH.E.highbeta.b.FP2.q.T6', 'COH.E.highbeta.b.FP2.r.O1', 'COH.E.highbeta.c.F7.d.F3', 'COH.E.highbeta.c.F7.e.Fz', 'COH.E.highbeta.c.F7.f.F4', 'COH.E.highbeta.c.F7.g.F8', 'COH.E.highbeta.c.F7.h.T3', 'COH.E.highbeta.c.F7.i.C3', 'COH.E.highbeta.c.F7.j.Cz', 'COH.E.highbeta.c.F7.k.C4', 'COH.E.highbeta.c.F7.l.T4', 'COH.E.highbeta.c.F7.m.T5', 'COH.E.highbeta.c.F7.n.P3', 'COH.E.highbeta.c.F7.r.O1', 'COH.E.highbeta.d.F3.e.Fz', 'COH.E.highbeta.d.F3.f.F4', 'COH.E.highbeta.d.F3.g.F8', 'COH.E.highbeta.d.F3.h.T3', 'COH.E.highbeta.d.F3.i.C3', 'COH.E.highbeta.d.F3.j.Cz', 'COH.E.highbeta.d.F3.k.C4', 'COH.E.highbeta.d.F3.n.P3', 'COH.E.highbeta.e.Fz.f.F4', 'COH.E.highbeta.e.Fz.g.F8', 'COH.E.highbeta.e.Fz.h.T3', 'COH.E.highbeta.e.Fz.i.C3', 'COH.E.highbeta.e.Fz.j.Cz', 'COH.E.highbeta.e.Fz.k.C4', 'COH.E.highbeta.e.Fz.l.T4', 'COH.E.highbeta.e.Fz.m.T5', 'COH.E.highbeta.e.Fz.r.O1', 'COH.E.highbeta.f.F4.g.F8', 'COH.E.highbeta.f.F4.h.T3', 'COH.E.highbeta.f.F4.i.C3', 'COH.E.highbeta.f.F4.j.Cz', 'COH.E.highbeta.f.F4.k.C4', 'COH.E.highbeta.f.F4.m.T5', 'COH.E.highbeta.f.F4.n.P3', 'COH.E.highbeta.f.F4.q.T6', 'COH.E.highbeta.f.F4.s.O2', 'COH.E.highbeta.g.F8.h.T3', 'COH.E.highbeta.g.F8.i.C3', 'COH.E.highbeta.g.F8.j.Cz', 'COH.E.highbeta.g.F8.k.C4', 'COH.E.highbeta.g.F8.l.T4', 'COH.E.highbeta.g.F8.m.T5', 'COH.E.highbeta.g.F8.n.P3', 'COH.E.highbeta.g.F8.o.Pz', 'COH.E.highbeta.g.F8.q.T6', 'COH.E.highbeta.g.F8.r.O1', 'COH.E.highbeta.g.F8.s.O2', 'COH.E.highbeta.h.T3.i.C3', 'COH.E.highbeta.h.T3.j.Cz', 'COH.E.highbeta.h.T3.l.T4', 'COH.E.highbeta.h.T3.m.T5', 'COH.E.highbeta.h.T3.n.P3', 'COH.E.highbeta.h.T3.o.Pz', 'COH.E.highbeta.h.T3.p.P4', 'COH.E.highbeta.h.T3.q.T6', 'COH.E.highbeta.h.T3.r.O1', 'COH.E.highbeta.h.T3.s.O2', 'COH.E.highbeta.i.C3.j.Cz', 'COH.E.highbeta.i.C3.k.C4', 'COH.E.highbeta.i.C3.l.T4', 'COH.E.highbeta.i.C3.m.T5', 'COH.E.highbeta.i.C3.n.P3', 'COH.E.highbeta.i.C3.o.Pz', 'COH.E.highbeta.i.C3.p.P4', 'COH.E.highbeta.i.C3.r.O1', 'COH.E.highbeta.j.Cz.k.C4', 'COH.E.highbeta.j.Cz.m.T5', 'COH.E.highbeta.j.Cz.n.P3', 'COH.E.highbeta.j.Cz.o.Pz', 'COH.E.highbeta.j.Cz.p.P4', 'COH.E.highbeta.j.Cz.q.T6', 'COH.E.highbeta.j.Cz.r.O1', 'COH.E.highbeta.j.Cz.s.O2', 'COH.E.highbeta.k.C4.l.T4', 'COH.E.highbeta.k.C4.m.T5', 'COH.E.highbeta.k.C4.n.P3', 'COH.E.highbeta.k.C4.o.Pz', 'COH.E.highbeta.k.C4.p.P4', 'COH.E.highbeta.k.C4.q.T6', 'COH.E.highbeta.k.C4.r.O1', 'COH.E.highbeta.k.C4.s.O2', 'COH.E.highbeta.l.T4.m.T5', 'COH.E.highbeta.l.T4.n.P3', 'COH.E.highbeta.l.T4.p.P4', 'COH.E.highbeta.l.T4.q.T6', 'COH.E.highbeta.l.T4.r.O1', 'COH.E.highbeta.l.T4.s.O2', 'COH.E.highbeta.m.T5.n.P3', 'COH.E.highbeta.m.T5.o.Pz', 'COH.E.highbeta.m.T5.p.P4', 'COH.E.highbeta.m.T5.q.T6', 'COH.E.highbeta.m.T5.r.O1', 'COH.E.highbeta.m.T5.s.O2', 'COH.E.highbeta.n.P3.o.Pz', 'COH.E.highbeta.n.P3.p.P4', 'COH.E.highbeta.n.P3.q.T6', 'COH.E.highbeta.n.P3.r.O1', 'COH.E.highbeta.n.P3.s.O2', 'COH.E.highbeta.o.Pz.p.P4', 'COH.E.highbeta.o.Pz.q.T6', 'COH.E.highbeta.o.Pz.r.O1', 'COH.E.highbeta.o.Pz.s.O2', 'COH.E.highbeta.p.P4.q.T6', 'COH.E.highbeta.p.P4.r.O1', 'COH.E.highbeta.p.P4.s.O2', 'COH.E.highbeta.q.T6.r.O1', 'COH.E.highbeta.q.T6.s.O2', 'COH.E.highbeta.r.O1.s.O2', 'COH.F.gamma.a.FP1.g.F8', 'COH.F.gamma.a.FP1.h.T3', 'COH.F.gamma.a.FP1.i.C3', 'COH.F.gamma.a.FP1.l.T4', 'COH.F.gamma.a.FP1.m.T5', 'COH.F.gamma.a.FP1.n.P3', 'COH.F.gamma.a.FP1.r.O1', 'COH.F.gamma.a.FP1.s.O2', 'COH.F.gamma.b.FP2.c.F7', 'COH.F.gamma.b.FP2.h.T3', 'COH.F.gamma.b.FP2.i.C3', 'COH.F.gamma.b.FP2.m.T5', 'COH.F.gamma.c.F7.e.Fz', 'COH.F.gamma.c.F7.l.T4', 'COH.F.gamma.d.F3.g.F8', 'COH.F.gamma.g.F8.h.T3', 'COH.F.gamma.g.F8.m.T5', 'COH.F.gamma.g.F8.n.P3', 'COH.F.gamma.g.F8.r.O1', 'COH.F.gamma.g.F8.s.O2', 'COH.F.gamma.h.T3.l.T4', 'COH.F.gamma.h.T3.r.O1', 'COH.F.gamma.h.T3.s.O2', 'COH.F.gamma.i.C3.m.T5', 'COH.F.gamma.i.C3.r.O1', 'COH.F.gamma.l.T4.m.T5', 'COH.F.gamma.l.T4.r.O1']


for i in range(len(arr)-1):                            # taking into consideration all the variables for clustering
    kmeans = KMeans(n_clusters = 2, random_state = 45)
    df_n['cluster'] = kmeans.fit_predict(df_n[[arr[i],arr[i+1]]])
#Getting the centroids
    centroids = kmeans.cluster_centers_
    cen_x = [i[0] for i in centroids] 
    cen_y = [i[1] for i in centroids]
    
    #Plotting the total numbers of elements in each cluster
    count = df_n['cluster'].value_counts()
    fig = plt.figure()
    ax = fig.add_axes([1,1,1,1])
    ax.bar(count.index,count)
    plt.xticks(count.index, color='black', fontweight='bold', fontsize='10', horizontalalignment='right')
    plt.xlabel('Clusters')
    plt.ylabel('Counts')
    plt.title('Cluster Counts')
    plt.show()
 #Adding the centroids to the dataframe
    df_n['cen_x'] = df_n.cluster.map({0:cen_x[0], 1:cen_x[1]})
    df_n['cen_y'] = df_n.cluster.map({0:cen_y[0], 1:cen_y[1]})
    #Defining the colur map
    colors = ['#DF2020', '#81DF20']
    df_n['c'] = df_n.cluster.map({0:colors[0], 1:colors[1]}) 
    #Plotting the scatter plot
    plt.scatter(df_n[arr[i]], df_n[arr[i+1]], c=df_n.c, alpha = 0.6, s=10)
    plt.title("scatter plot")
    plt.xlabel(arr[i])
    plt.ylabel(arr[i+1])

### Machine learning modelling

In [ ]:
# Splitting the data into X_train, X_test, y_train and y_test using SciKit-learn with test size as 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

### K-Nearest Neighbour classifier

In [ ]:
# Importing the required libararies for Cross Validation
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# With the help of 10-Fold Cross Validation building k-nearest neighbors classifier with shuffled data
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = KNeighborsClassifier()

# Predicting accuracy for each of the Cross Validation
scores1 = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print("Accuracy of the model with 10 K-Fold Cross Validation:")
print(scores1)

##### CALCULATING THE FIRST PERFORMANCE METRIC i.e. RECALL

In [ ]:
# Calculating first performance metric i.e recall

scores1_recall = cross_val_score(model, X, y, scoring='recall_weighted', cv=cv, n_jobs=-1)
print("Recall for the model with 10 K-Fold Cross Validation:")
print(scores1_recall)

##### CALCULATING THE SECOND PERFORMANCE METRIC i.e. ACCURACY ALONG WITH CONFUSION MATRIX

In [ ]:
# Creating an object for the classifier
knn = KNeighborsClassifier()

# Fitting the model
knn.fit(X_train, y_train)
prediction_train=knn.predict(X_train)
# Predicting the output
prediction = knn.predict(X_test)

# Calling the required library for outputting the accuracy, confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

# Outputting the first performance metric i.e. Accuracy
print("First Performance Metric")
print("Accuracy of the K-Nearest Neighbors Classifier using train data:", accuracy_score(y_train, prediction_train), "\n")
print("Accuracy of the K-Nearest Neighbors Classifier:", accuracy_score(y_test, prediction), "\n")

# Outputting the second performance metric i.e. Confusion Matrix
print("Second Performance Metric")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction), "\n")

# Outputting classification report containing Recall and Precision
print("Classification Report:\n", classification_report(y_test, prediction))

### Random Forest classifier

In [ ]:
# With the help of 10-Fold Cross Validation building random forest classifier with shuffled data
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = RandomForestClassifier()

# Predicting accuracy for each of the Cross Validation
scores2 = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print("Accuracy of the model with 10 K-Fold Cross Validation:")
print(scores2)

##### CALCULATING THE FIRST PERFORMANCE METRIC i.e. RECALL

In [ ]:
#Calculating first performance metric i.e. Recall

scores2_recall = cross_val_score(model, X, y, scoring='recall_weighted', cv=cv, n_jobs=-1)
print("Recall for the model with 10 K-Fold Cross Validation:")
print(scores2_recall)

##### CALCULATING THE SECOND PERFORMANCE METRIC i.e. ACCURACY ALONG WITH CONFUSION MATRIX

In [ ]:
# Creating an object for the classifier
rand_forest = RandomForestClassifier()

# Fitting the model
rand_forest.fit(X_train, y_train)
prediction_train = rand_forest.predict(X_train)
# Predicting the output
prediction = rand_forest.predict(X_test)

# Outputting the first performance metric i.e. Accuracy
print("First Performance Metric")
accuracy1 = accuracy_score(y_train, prediction_train)
accuracy = accuracy_score(y_test, prediction)
print("Accuracy of the Random Forest Classifier using train data:", accuracy1, "\n")
print("Accuracy of the Random Forest Classifier:", accuracy, "\n")

# Outputting the second performance metric i.e. Confusion Matrix
print("Second Performance Metric")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction), "\n")

# Outputting classification report containing Recall and Precision
print("Classification Report:\n", classification_report(y_test, prediction))

##### IMPROVISATION IN RANDOM FOREST CLASSIFIER IN ORDER TO IMPROVE ACCURACY

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3-fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=1, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
# Get the best parameters
rf_random.best_params_

In [ ]:
# Get the Random Forest Classifier with best parameters and store in an object
sum = 0 
best_random = rf_random.best_estimator_
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Predicting accuracy for each of the Cross Validation
scores2_new = cross_val_score(best_random, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# Displays the accuracy of the Random Forest Classifier with the best parameters using RandomisedSearchCV
# Moreover, using 10-fold model validation along with the above
sum = sum + scores2_new
print(scores2_new, "\n")
print(sum/10)

### SVM classifier

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
#train SVM model
from sklearn import svm
# With the help of 10-Fold Cross Validation building random forest classifier with shuffled data
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = svm.SVC(kernel = 'linear')

# Predicting accuracy for each of the Cross Validation
scores2 = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print("Accuracy of the model with 10 K-Fold Cross Validation:")
print(scores2)

In [ ]:
# CALCULATING THE FIRST PERFORMANCE METRIC i.e. RECALL
scores2_recall = cross_val_score(model, X, y, scoring='recall_weighted', cv=cv, n_jobs=-1)
print("Recall for the model with 10 K-Fold Cross Validation:")
print(scores2_recall)

In [ ]:
# Calculating accuracy

from sklearn.svm import SVC
from sklearn import metrics
#train SVM model
from sklearn import svm
clf = svm.SVC(kernel = 'linear')
clf.fit(X_train, y_train)
#store predictions and ground truth
y_pred = clf.predict(X_train)
y_true = y_train

#assess the performance of the SVM with linear kernel on Training data
print('linear kernel Accuracy with train : ', metrics.accuracy_score(y_true, y_pred))

#Now, use the SVM model to predict Test data
y_pred = clf.predict(X_test)
y_true = y_test

#assess the performance of the SVM with linear kernel on Testing data
print('linear kernel Accuracy with test : ', metrics.accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
#train SVM model
from sklearn import svm
# With the help of 10-Fold Cross Validation building random forest classifier with shuffled data
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = svm.SVC(kernel = 'rbf')

# Predicting accuracy for each of the Cross Validation
scores2 = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print("Accuracy of the model with 10 K-Fold Cross Validation:")
print(scores2)

In [ ]:
# CALCULATING THE FIRST PERFORMANCE METRIC i.e. RECALL
scores2_recall = cross_val_score(model, X, y, scoring='recall_weighted', cv=cv, n_jobs=-1)
print("Recall for the model with 10 K-Fold Cross Validation:")
print(scores2_recall)

In [ ]:
#Train a SVM using RBF kernel
clf = svm.SVC(kernel = 'rbf')
clf.fit(X_train, y_train)

#store predictions and ground truth
y_pred = clf.predict(X_train)
y_true = y_train

#assess the performance of the SVM with linear kernel on Training data
print('RBF Accuracy with train data : ', metrics.accuracy_score(y_true, y_pred))

#Now, use the SVM model to predict Test data
y_pred = clf.predict(X_test)
y_true = y_test

#assess the performance of the SVM with linear kernel on Testing data
print('RBF Accuracy with test : ', metrics.accuracy_score(y_true, y_pred))